### This is Luke's Final Code Deliverable for D4D .

User selects a date from the combination of arrival and departure dates, which are loaded into own dataframe that is loaded into a pyplot graph(or heat map) or Folium map. This will be a standalone executable with user using the command line.

**FUNCTIONALITY**

User is prompted to enter a month. 'Please enter the first 3 letters of the desired month'
A list of Available dates in xxx is printed 1,2,...31
User is prompted to enter a month. 'Please enter the number of the desired day'

User chosen date is processed. This means for the particular date, all the geometries of arrival & departures are gathered. 
After gathered they are color coded and loaded into the map.


Once we can do that we should be able to display arrival and departures data in different colors in a Folium map that appears in a browser. A particular date would display arrival data in green and departure data in say red.

In [1]:
import pandas as pd
import numpy as np 
import geopandas as gpd
import scipy.stats as stats
import math
import matplotlib.pylab as plt
%matplotlib inline
import os
import folium
from sklearn.cluster import KMeans

In [2]:
dates_dict_2018 = {'nov': ['13', '01', '14', '02', '15', '03', '16', '04', '17', '05', '18', '06', '19', '07', '20', '08', '21', '09', '22', '10', '23', '11', '24', '12', '25', '26', '27', '28', '29', '30'], 'dec': ['01', '02', '03', '04', '05', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '06', '19', '21', '22', '23', '24', '25', '29', '30', '31', '20', '26', '27', '28']}
dates_dict_2019={'jan': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '24', '26', '27', '28', '16', '29', '17', '30', '31', '23'], 'feb': ['01', '02', '03', '04', '07', '08', '09', '10', '11', '13', '14', '15', '16', '19', '20', '21', '22', '18', '24', '25', '26', '27', '28'], 'mar': ['02', '03', '04', '05', '06', '08', '09', '10', '11', '12', '13', '01', '15', '16', '17', '18', '21', '23', '25', '27', '28', '29', '30', '31', '19', '20', '22', '24'], 'apr': ['01', '02', '03', '04', '05', '08', '10', '11', '17', '20', '21', '09', '22', '23', '25', '13', '27', '15', '28', '16', '29', '24', '26'], 'may': ['02', '03', '04', '06', '08', '14', '15', '16', '17', '05', '18', '19', '07', '21', '09', '22', '10', '23', '11', '24', '12', '26', '27', '28', '29', '30', '31', '20'], 'jun': ['01', '02', '03', '04', '05', '07', '08', '09', '10', '11', '14', '15', '17', '18', '06', '21', '23', '24', '12', '25', '13', '28', '16', '29', '30', '19', '22', '26', '27', '20'], 'jul': ['07', '08', '09', '10', '11', '01', '13', '02', '14', '15', '03', '16', '04', '17', '05', '06', '18', '19', '20', '21', '22', '24', '12', '26', '27', '28', '29', '30', '31', '23', '25'], 'aug': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '12', '13', '14', '15', '16', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '17', '31', '23'], 'sep': ['02', '03', '04', '05', '06', '07', '08', '10', '12', '13', '01', '14', '15', '16', '17', '18', '19', '20', '21', '09', '22', '23', '11', '25', '30', '24', '26', '28'], 'oct': ['02', '04', '05', '06', '07', '08', '10', '17', '18', '19', '20', '22', '23', '11', '25', '13', '26', '14', '27', '15', '28', '16', '30']} 

In [3]:
month_list = ('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov','dec')
day_list = ()

user_month = ''
valid_input = False
while not valid_input:
    if user_month in month_list:
        valid_input = True
    else:
        print('Please enter a valid month using the first 3 letters in lowercase (such as jan):')
        user_month = input()
        
for k in dates_dict_2018:
    if user_month == k:
        day_list = dates_dict_2018[user_month]
        print(day_list)
for l in dates_dict_2019:
    if user_month == l:
        day_list = dates_dict_2019[user_month]
        print(day_list)
        
user_day = ''
valid_input = False
while not valid_input:
    if user_day in day_list:
        valid_input = True
    else:
        print('From the List above, please enter number of the desired date (such as 17):')
        user_day = input()
        
print(user_month + " " + user_day + " is being processed")

Please enter a valid month using the first 3 letters in lowercase (such as jan):
apr
['01', '02', '03', '04', '05', '08', '10', '11', '17', '20', '21', '09', '22', '23', '25', '13', '27', '15', '28', '16', '29', '24', '26']
From the List above, please enter number of the desired date (such as 17):
21
apr 21 is being processed


In [4]:
month_file_dict = {'jan': ['20181220025446_20190101025446_33160N-ACD-BETA',
  '20181221024550_20190102024549_33341N-ACD-BETA',
  '20181221024611_20190102024610_314013N-ACD-BETA',
  '20181221152404_20190102152403_322516N-ACD-BETA',
  '20181221152429_20190102152429_34124N-ACD-BETA',
  '20181222151621_20190103151620_314913N-ACD-BETA',
  '20181222151646_20190103151645_332036N-ACD-BETA',
  '20181222151710_20190103151710_345013N-ACD-BETA',
  '20181223150644_20190104150644_292132N-ACD-BETA',
  '20181223150710_20190104150709_305244N-ACD-BETA',
  '20181223150735_20190104150734_32248N-ACD-BETA',
  '20181223150759_20190104150759_335348N-ACD-BETA',
  '20181223150826_20190104150826_352655N-ACD-BETA',
  '20181223150902_20190104150902_373457N-ACD-BETA',
  '20181224030906_20190105030905_381041N-ACD-BETA',
  '20181224030930_20190105030930_364057N-ACD-BETA',
  '20181224030955_20190105030955_351116N-ACD-BETA',
  '20181224031021_20190105031021_333951N-ACD-BETA',
  '20181225030222_20190106030221_351851N-ACD-BETA',
  '20181225145215_20190106145215_364747N-ACD-BETA',
  '20181226025306_20190107025306_363434N-ACD-BETA',
  '20181226025332_20190107025331_345831N-ACD-BETA',
  '20181226025357_20190107025356_33273N-ACD-BETA',
  '20181226025429_20190107025428_314346N-ACD-BETA',
  '20181227024632_20190108024631_33016N-ACD-BETA',
  '20181227024657_20190108024656_313021N-ACD-BETA',
  '20181228151454_20190109151453_2935N-ACD-BETA',
  '20181228151520_20190109151519_303917N-ACD-BETA',
  '20181228151544_20190109151543_32913N-ACD-BETA',
  '20181228151619_20190109151619_341212N-ACD-BETA',
  '20181228151645_20190109151645_354816N-ACD-BETA',
  '20181229031743_20190110031742_364428N-ACD-BETA',
  '20181229031808_20190110031807_351451N-ACD-BETA',
  '20181229031834_20190110031833_334329N-ACD-BETA',
  '20181229031859_20190110031859_321021N-ACD-BETA',
  '20181230030955_20190111030954_374540N-ACD-BETA',
  '20181230031019_20190111031019_361419N-ACD-BETA',
  '20181230031044_20190111031044_344440N-ACD-BETA',
  '20181230031109_20190111031108_33151N-ACD-BETA',
  '20181230031131_20190111031131_315021N-ACD-BETA',
  '20181230145919_20190111145919_321328N-ACD-BETA',
  '20181230145945_20190111145944_334936N-ACD-BETA',
  '20181230150009_20190111150009_352058N-ACD-BETA',
  '20181230150034_20190111150034_365035N-ACD-BETA',
  '20181230150059_20190111150059_382023N-ACD-BETA',
  '20181231145103_20190112145102_293910N-ACD-BETA',
  '20181231145129_20190112145128_311038N-ACD-BETA',
  '20181231145154_20190112145153_324153N-ACD-BETA',
  '20181231145218_20190112145218_341141N-ACD-BETA',
  '20181231145243_20190112145243_354119N-ACD-BETA',
  '20181231145308_20190112145307_371056N-ACD-BETA',
  '20190101025355_20190113025354_3634N-ACD-BETA',
  '20190101025420_20190113025419_343327N-ACD-BETA',
  '20190101025446_20190113025445_33158N-ACD-BETA',
  '20190102024549_20190114024549_33341N-ACD-BETA',
  '20190102024610_20190114024610_314013N-ACD-BETA',
  '20190103151555_20190115151554_30133N-ACD-BETA',
  '20190106030155_20190118030155_365157N-ACD-BETA',
  '20190106030221_20190118030221_351849N-ACD-BETA',
  '20190106145109_20190118145108_322656N-ACD-BETA',
  '20190106145134_20190118145134_34314N-ACD-BETA',
  '20190106145159_20190118145159_353433N-ACD-BETA',
  '20190106145215_20190118145215_364747N-ACD-BETA',
  '20190107144245_20190119144245_292631N-ACD-BETA',
  '20190107144311_20190119144311_30581N-ACD-BETA',
  '20190108024631_20190120024631_33015N-ACD-BETA',
  '20190108024656_20190120024656_313020N-ACD-BETA',
  '20190108024721_20190120024721_30036N-ACD-BETA',
  '20190108024744_20190120024744_28343N-ACD-BETA',
  '20190108152407_20190120152407_301212N-ACD-BETA',
  '20190108152432_20190120152432_314338N-ACD-BETA',
  '20190108152457_20190120152456_331319N-ACD-BETA',
  '20190108152521_20190120152521_344257N-ACD-BETA',
  '20190109151543_20190121151543_32913N-ACD-BETA',
  '20190109151619_20190121151619_341212N-ACD-BETA',
  '20190109151645_20190121151644_354817N-ACD-BETA',
  '20190110031718_20190122031717_381411N-ACD-BETA',
  '20190110031742_20190122031742_364428N-ACD-BETA',
  '20190110031807_20190122031807_351451N-ACD-BETA',
  '20190110031833_20190122031833_334328N-ACD-BETA',
  '20190110031859_20190122031858_321021N-ACD-BETA',
  '20190110150812_20190122150812_32614N-ACD-BETA',
  '20190110150838_20190122150837_334223N-ACD-BETA',
  '20190110150902_20190122150902_351344N-ACD-BETA',
  '20190110150927_20190122150927_364320N-ACD-BETA',
  '20190110150952_20190122150952_381256N-ACD-BETA',
  '20190112145102_20190124145102_29399N-ACD-BETA',
  '20190112145128_20190124145128_311037N-ACD-BETA',
  '20190112145153_20190124145153_324152N-ACD-BETA',
  '20190112145218_20190124145218_341141N-ACD-BETA',
  '20190112145243_20190124145242_354120N-ACD-BETA',
  '20190112145307_20190124145307_371056N-ACD-BETA',
  '20190114024549_20190126024548_33342N-ACD-BETA',
  '20190114024610_20190126024610_314014N-ACD-BETA',
  '20190114152324_20190126152324_295712N-ACD-BETA',
  '20190114152403_20190126152403_322516N-ACD-BETA',
  '20190114152429_20190126152428_34124N-ACD-BETA',
  '20190115032625_20190127032625_35031N-ACD-BETA',
  '20190115032650_20190127032650_332911N-ACD-BETA',
  '20190115032715_20190127032715_315931N-ACD-BETA',
  '20190115151554_20190127151554_30133N-ACD-BETA',
  '20190115151620_20190127151620_314915N-ACD-BETA',
  '20190115151645_20190127151644_332037N-ACD-BETA',
  '20190115151710_20190127151709_345014N-ACD-BETA',
  '20190115151734_20190127151734_361952N-ACD-BETA',
  '20190116031807_20190128031806_37498N-ACD-BETA',
  '20190116031832_20190128031831_361753N-ACD-BETA',
  '20190116031856_20190128031856_344816N-ACD-BETA',
  '20190116150643_20190128150643_292133N-ACD-BETA',
  '20190116150709_20190128150709_305245N-ACD-BETA',
  '20190116150734_20190128150733_32249N-ACD-BETA',
  '20190116150759_20190128150758_335350N-ACD-BETA',
  '20190116150825_20190128150825_352657N-ACD-BETA',
  '20190117030905_20190129030904_381041N-ACD-BETA',
  '20190117030954_20190129030954_351116N-ACD-BETA',
  '20190117031020_20190129031020_333951N-ACD-BETA',
  '20190117145946_20190129145946_312832N-ACD-BETA',
  '20190117150011_20190129150011_325813N-ACD-BETA',
  '20190117150036_20190129150036_342752N-ACD-BETA',
  '20190117150101_20190129150100_355731N-ACD-BETA',
  '20190117150127_20190129150126_372846N-ACD-BETA',
  '20190118030155_20190130030155_365156N-ACD-BETA',
  '20190118145108_20190130145108_322656N-ACD-BETA',
  '20190118145134_20190130145134_34314N-ACD-BETA',
  '20190118145215_20190130145214_364748N-ACD-BETA',
  '20190119025305_20190131025305_363434N-ACD-BETA',
  '20190119025331_20190131025331_345832N-ACD-BETA',
  '20190119025356_20190131025355_33274N-ACD-BETA',
  '20190119025428_20190131025428_314347N-ACD-BETA',
  '20190119144245_20190131144245_292629N-ACD-BETA',
  '20190120024631_20190201024631_33015N-ACD-BETA',
  '20190120024656_20190201024656_313020N-ACD-BETA',
  '20190120024721_20190201024721_30036N-ACD-BETA',
  '20190120024744_20190201024744_28344N-ACD-BETA',
  '20190120152407_20190201152406_301211N-ACD-BETA',
  '20190120152432_20190201152431_314337N-ACD-BETA',
  '20190120152456_20190201152456_331319N-ACD-BETA',
  '20190120152521_20190201152521_344257N-ACD-BETA',
  '20190121032711_20190202032710_344534N-ACD-BETA',
  '20190121032735_20190202032735_331558N-ACD-BETA',
  '20190121032800_20190202032800_314618N-ACD-BETA',
  '20190121151453_20190202151453_2935N-ACD-BETA',
  '20190121151519_20190202151518_303917N-ACD-BETA',
  '20190121151543_20190202151542_32913N-ACD-BETA',
  '20190121151619_20190202151618_341212N-ACD-BETA',
  '20190121151644_20190202151644_354817N-ACD-BETA',
  '20190122031717_20190203031717_381411N-ACD-BETA',
  '20190122031742_20190203031742_364427N-ACD-BETA',
  '20190122031807_20190203031807_351450N-ACD-BETA',
  '20190122031833_20190203031832_334327N-ACD-BETA',
  '20190122031858_20190203031858_321020N-ACD-BETA',
  '20190122150812_20190203150811_32613N-ACD-BETA',
  '20190122150837_20190203150837_334222N-ACD-BETA',
  '20190122150927_20190203150927_364319N-ACD-BETA',
  '20190123145918_20190204145918_321328N-ACD-BETA',
  '20190123145944_20190204145944_334936N-ACD-BETA',
  '20190123150009_20190204150008_352058N-ACD-BETA',
  '20190123150033_20190204150033_365036N-ACD-BETA',
  '20190123150058_20190204150058_382023N-ACD-BETA',
  '20190126152428_20190207152428_34124N-ACD-BETA',
  '20190127023903_20190208023903_301317N-ACD-BETA',
  '20190128031806_20190209031806_37498N-ACD-BETA',
  '20190128031831_20190209031831_361753N-ACD-BETA',
  '20190128031856_20190209031855_344816N-ACD-BETA',
  '20190128031921_20190209031920_331839N-ACD-BETA',
  '20190128150643_20190209150642_292131N-ACD-BETA',
  '20190128150709_20190209150708_305243N-ACD-BETA',
  '20190128150733_20190209150733_32248N-ACD-BETA',
  '20190128150758_20190209150758_335348N-ACD-BETA',
  '20190128150825_20190209150825_352655N-ACD-BETA',
  '20190128150901_20190209150900_373457N-ACD-BETA',
  '20190129030904_20190210030904_381040N-ACD-BETA',
  '20190129030929_20190210030929_364056N-ACD-BETA',
  '20190129030954_20190210030954_351116N-ACD-BETA',
  '20190129031020_20190210031019_333952N-ACD-BETA',
  '20190129031051_20190210031051_315647N-ACD-BETA',
  '20190129150011_20190210150010_325812N-ACD-BETA',
  '20190129150036_20190210150035_342751N-ACD-BETA',
  '20190129150100_20190210150100_355730N-ACD-BETA',
  '20190129150126_20190210150126_372846N-ACD-BETA',
  '20190130145108_20190211145108_322656N-ACD-BETA',
  '20190130145134_20190211145133_34313N-ACD-BETA',
  '20190130145159_20190211145158_353432N-ACD-BETA',
  '20190130145214_20190211145214_364747N-ACD-BETA'],
 'feb': ['20190120024631_20190201024631_33015N-ACD-BETA',
  '20190120024656_20190201024656_313020N-ACD-BETA',
  '20190120024721_20190201024721_30036N-ACD-BETA',
  '20190120024744_20190201024744_28344N-ACD-BETA',
  '20190120152407_20190201152406_301211N-ACD-BETA',
  '20190120152432_20190201152431_314337N-ACD-BETA',
  '20190120152456_20190201152456_331319N-ACD-BETA',
  '20190120152521_20190201152521_344257N-ACD-BETA',
  '20190121032711_20190202032710_344534N-ACD-BETA',
  '20190121032735_20190202032735_331558N-ACD-BETA',
  '20190121032800_20190202032800_314618N-ACD-BETA',
  '20190121151453_20190202151453_2935N-ACD-BETA',
  '20190121151519_20190202151518_303917N-ACD-BETA',
  '20190121151543_20190202151542_32913N-ACD-BETA',
  '20190121151619_20190202151618_341212N-ACD-BETA',
  '20190121151644_20190202151644_354817N-ACD-BETA',
  '20190122031717_20190203031717_381411N-ACD-BETA',
  '20190122031742_20190203031742_364427N-ACD-BETA',
  '20190122031807_20190203031807_351450N-ACD-BETA',
  '20190122031833_20190203031832_334327N-ACD-BETA',
  '20190122031858_20190203031858_321020N-ACD-BETA',
  '20190122150812_20190203150811_32613N-ACD-BETA',
  '20190122150837_20190203150837_334222N-ACD-BETA',
  '20190122150927_20190203150927_364319N-ACD-BETA',
  '20190123145918_20190204145918_321328N-ACD-BETA',
  '20190123145944_20190204145944_334936N-ACD-BETA',
  '20190123150009_20190204150008_352058N-ACD-BETA',
  '20190123150033_20190204150033_365036N-ACD-BETA',
  '20190123150058_20190204150058_382023N-ACD-BETA',
  '20190126152428_20190207152428_34124N-ACD-BETA',
  '20190127023903_20190208023903_301317N-ACD-BETA',
  '20190128031806_20190209031806_37498N-ACD-BETA',
  '20190128031831_20190209031831_361753N-ACD-BETA',
  '20190128031856_20190209031855_344816N-ACD-BETA',
  '20190128031921_20190209031920_331839N-ACD-BETA',
  '20190128150643_20190209150642_292131N-ACD-BETA',
  '20190128150709_20190209150708_305243N-ACD-BETA',
  '20190128150733_20190209150733_32248N-ACD-BETA',
  '20190128150758_20190209150758_335348N-ACD-BETA',
  '20190128150825_20190209150825_352655N-ACD-BETA',
  '20190128150901_20190209150900_373457N-ACD-BETA',
  '20190129030904_20190210030904_381040N-ACD-BETA',
  '20190129030929_20190210030929_364056N-ACD-BETA',
  '20190129030954_20190210030954_351116N-ACD-BETA',
  '20190129031020_20190210031019_333952N-ACD-BETA',
  '20190129031051_20190210031051_315647N-ACD-BETA',
  '20190129150011_20190210150010_325812N-ACD-BETA',
  '20190129150036_20190210150035_342751N-ACD-BETA',
  '20190129150100_20190210150100_355730N-ACD-BETA',
  '20190129150126_20190210150126_372846N-ACD-BETA',
  '20190130145108_20190211145108_322656N-ACD-BETA',
  '20190130145134_20190211145133_34313N-ACD-BETA',
  '20190130145159_20190211145158_353432N-ACD-BETA',
  '20190130145214_20190211145214_364747N-ACD-BETA',
  '20190201152406_20190213152406_301210N-ACD-BETA',
  '20190201152431_20190213152431_314336N-ACD-BETA',
  '20190201152456_20190213152456_331317N-ACD-BETA',
  '20190201152521_20190213152520_344255N-ACD-BETA',
  '20190202151453_20190214151452_2935N-ACD-BETA',
  '20190202151518_20190214151518_303917N-ACD-BETA',
  '20190202151542_20190214151542_32913N-ACD-BETA',
  '20190202151618_20190214151618_341213N-ACD-BETA',
  '20190202151644_20190214151643_354817N-ACD-BETA',
  '20190203150811_20190215150811_32613N-ACD-BETA',
  '20190203150837_20190215150837_334222N-ACD-BETA',
  '20190203150902_20190215150901_351342N-ACD-BETA',
  '20190203150927_20190215150926_364319N-ACD-BETA',
  '20190204145918_20190216145918_321327N-ACD-BETA',
  '20190204145944_20190216145943_334935N-ACD-BETA',
  '20190204150008_20190216150008_352058N-ACD-BETA',
  '20190204150033_20190216150033_365036N-ACD-BETA',
  '20190204150058_20190216150058_382024N-ACD-BETA',
  '20190207152428_20190219152428_34123N-ACD-BETA',
  '20190208151553_20190220151553_30133N-ACD-BETA',
  '20190208151644_20190220151644_332037N-ACD-BETA',
  '20190208151709_20190220151709_345014N-ACD-BETA',
  '20190208151733_20190220151733_361952N-ACD-BETA',
  '20190209150642_20190221150642_292131N-ACD-BETA',
  '20190209150708_20190221150708_305244N-ACD-BETA',
  '20190209150733_20190221150733_32249N-ACD-BETA',
  '20190209150758_20190221150758_335349N-ACD-BETA',
  '20190209150825_20190221150824_352656N-ACD-BETA',
  '20190210145855_20190222145855_282547N-ACD-BETA',
  '20190210150100_20190222150100_355732N-ACD-BETA',
  '20190210150126_20190222150126_372847N-ACD-BETA',
  '20190218025353_20190302025354_3635N-ACD-BETA',
  '20190218025418_20190302025418_343327N-ACD-BETA',
  '20190218025444_20190302025444_33158N-ACD-BETA',
  '20190219024548_20190303024548_33341N-ACD-BETA',
  '20190219024609_20190303024609_314013N-ACD-BETA',
  '20190219152323_20190303152323_295713N-ACD-BETA',
  '20190219152402_20190303152402_322516N-ACD-BETA',
  '20190219152428_20190303152428_34124N-ACD-BETA',
  '20190220023903_20190304023903_301315N-ACD-BETA',
  '20190220151553_20190304151553_30131N-ACD-BETA',
  '20190220151619_20190304151619_314913N-ACD-BETA',
  '20190220151644_20190304151644_332036N-ACD-BETA',
  '20190220151709_20190304151709_345013N-ACD-BETA',
  '20190220151733_20190304151734_361951N-ACD-BETA',
  '20190221150708_20190305150708_305243N-ACD-BETA',
  '20190221150733_20190305150733_32247N-ACD-BETA',
  '20190221150758_20190305150758_335348N-ACD-BETA',
  '20190221150824_20190305150824_352655N-ACD-BETA',
  '20190221150900_20190305150900_373457N-ACD-BETA',
  '20190222030929_20190306030929_364055N-ACD-BETA',
  '20190222145855_20190306145855_282545N-ACD-BETA',
  '20190222145921_20190306145921_295715N-ACD-BETA',
  '20190222145945_20190306145945_312830N-ACD-BETA',
  '20190222150010_20190306150010_325811N-ACD-BETA',
  '20190222150035_20190306150035_342750N-ACD-BETA',
  '20190222150100_20190306150100_355730N-ACD-BETA',
  '20190222150126_20190306150126_372846N-ACD-BETA',
  '20190224025330_20190308025330_345831N-ACD-BETA',
  '20190224025355_20190308025355_33274N-ACD-BETA',
  '20190224025427_20190308025427_314347N-ACD-BETA',
  '20190224144244_20190308144244_292629N-ACD-BETA',
  '20190224144310_20190308144310_305758N-ACD-BETA',
  '20190225152406_20190309152406_301210N-ACD-BETA',
  '20190225152431_20190309152431_314335N-ACD-BETA',
  '20190225152456_20190309152456_331316N-ACD-BETA',
  '20190225152520_20190309152520_344254N-ACD-BETA',
  '20190226151518_20190310151518_303915N-ACD-BETA',
  '20190226151542_20190310151542_32912N-ACD-BETA',
  '20190226151618_20190310151618_341212N-ACD-BETA',
  '20190226151644_20190310151643_354816N-ACD-BETA',
  '20190227031717_20190311031716_381411N-ACD-BETA',
  '20190227031741_20190311031741_364427N-ACD-BETA',
  '20190227031806_20190311031806_351451N-ACD-BETA',
  '20190227031832_20190311031832_334329N-ACD-BETA',
  '20190227031858_20190311031858_321021N-ACD-BETA',
  '20190227150811_20190311150811_32612N-ACD-BETA',
  '20190227150837_20190311150837_334221N-ACD-BETA',
  '20190227150901_20190311150901_351341N-ACD-BETA',
  '20190227150926_20190311150926_364318N-ACD-BETA',
  '20190227150951_20190311150951_381254N-ACD-BETA',
  '20190228030953_20190312030953_374540N-ACD-BETA',
  '20190228031018_20190312031018_361419N-ACD-BETA',
  '20190228031043_20190312031043_344440N-ACD-BETA',
  '20190228031108_20190312031107_33151N-ACD-BETA',
  '20190228031130_20190312031130_315021N-ACD-BETA'],
 'mar': ['20190218025353_20190302025354_3635N-ACD-BETA',
  '20190218025418_20190302025418_343327N-ACD-BETA',
  '20190218025444_20190302025444_33158N-ACD-BETA',
  '20190219024548_20190303024548_33341N-ACD-BETA',
  '20190219024609_20190303024609_314013N-ACD-BETA',
  '20190219152323_20190303152323_295713N-ACD-BETA',
  '20190219152402_20190303152402_322516N-ACD-BETA',
  '20190219152428_20190303152428_34124N-ACD-BETA',
  '20190220023903_20190304023903_301315N-ACD-BETA',
  '20190220151553_20190304151553_30131N-ACD-BETA',
  '20190220151619_20190304151619_314913N-ACD-BETA',
  '20190220151644_20190304151644_332036N-ACD-BETA',
  '20190220151709_20190304151709_345013N-ACD-BETA',
  '20190220151733_20190304151734_361951N-ACD-BETA',
  '20190221150708_20190305150708_305243N-ACD-BETA',
  '20190221150733_20190305150733_32247N-ACD-BETA',
  '20190221150758_20190305150758_335348N-ACD-BETA',
  '20190221150824_20190305150824_352655N-ACD-BETA',
  '20190221150900_20190305150900_373457N-ACD-BETA',
  '20190222030929_20190306030929_364055N-ACD-BETA',
  '20190222145855_20190306145855_282545N-ACD-BETA',
  '20190222145921_20190306145921_295715N-ACD-BETA',
  '20190222145945_20190306145945_312830N-ACD-BETA',
  '20190222150010_20190306150010_325811N-ACD-BETA',
  '20190222150035_20190306150035_342750N-ACD-BETA',
  '20190222150100_20190306150100_355730N-ACD-BETA',
  '20190222150126_20190306150126_372846N-ACD-BETA',
  '20190224025330_20190308025330_345831N-ACD-BETA',
  '20190224025355_20190308025355_33274N-ACD-BETA',
  '20190224025427_20190308025427_314347N-ACD-BETA',
  '20190224144244_20190308144244_292629N-ACD-BETA',
  '20190224144310_20190308144310_305758N-ACD-BETA',
  '20190225152406_20190309152406_301210N-ACD-BETA',
  '20190225152431_20190309152431_314335N-ACD-BETA',
  '20190225152456_20190309152456_331316N-ACD-BETA',
  '20190225152520_20190309152520_344254N-ACD-BETA',
  '20190226151518_20190310151518_303915N-ACD-BETA',
  '20190226151542_20190310151542_32912N-ACD-BETA',
  '20190226151618_20190310151618_341212N-ACD-BETA',
  '20190226151644_20190310151643_354816N-ACD-BETA',
  '20190227031717_20190311031716_381411N-ACD-BETA',
  '20190227031741_20190311031741_364427N-ACD-BETA',
  '20190227031806_20190311031806_351451N-ACD-BETA',
  '20190227031832_20190311031832_334329N-ACD-BETA',
  '20190227031858_20190311031858_321021N-ACD-BETA',
  '20190227150811_20190311150811_32612N-ACD-BETA',
  '20190227150837_20190311150837_334221N-ACD-BETA',
  '20190227150901_20190311150901_351341N-ACD-BETA',
  '20190227150926_20190311150926_364318N-ACD-BETA',
  '20190227150951_20190311150951_381254N-ACD-BETA',
  '20190228030953_20190312030953_374540N-ACD-BETA',
  '20190228031018_20190312031018_361419N-ACD-BETA',
  '20190228031043_20190312031043_344440N-ACD-BETA',
  '20190228031108_20190312031107_33151N-ACD-BETA',
  '20190228031130_20190312031130_315021N-ACD-BETA',
  '20190301145127_20190313145127_311035N-ACD-BETA',
  '20190301145152_20190313145152_324150N-ACD-BETA',
  '20190301145217_20190313145217_341138N-ACD-BETA',
  '20190301145242_20190313145242_354117N-ACD-BETA',
  '20190301145307_20190313145306_371053N-ACD-BETA',
  '20190303152323_20190315152323_295712N-ACD-BETA',
  '20190303152402_20190315152402_322515N-ACD-BETA',
  '20190303152428_20190315152427_34123N-ACD-BETA',
  '20190304151553_20190316151553_30131N-ACD-BETA',
  '20190304151619_20190316151619_314913N-ACD-BETA',
  '20190304151644_20190316151644_332036N-ACD-BETA',
  '20190304151709_20190316151709_345013N-ACD-BETA',
  '20190304151734_20190316151733_361951N-ACD-BETA',
  '20190305150642_20190317150642_292130N-ACD-BETA',
  '20190305150708_20190317150708_305242N-ACD-BETA',
  '20190305150733_20190317150733_32246N-ACD-BETA',
  '20190305150758_20190317150758_335347N-ACD-BETA',
  '20190305150824_20190317150824_352654N-ACD-BETA',
  '20190305150900_20190317150900_373457N-ACD-BETA',
  '20190306031050_20190318031050_315647N-ACD-BETA',
  '20190309152406_20190321152406_301210N-ACD-BETA',
  '20190309152520_20190321152520_344254N-ACD-BETA',
  '20190311031716_20190323031716_381412N-ACD-BETA',
  '20190311031741_20190323031741_364428N-ACD-BETA',
  '20190311031806_20190323031806_351450N-ACD-BETA',
  '20190311031832_20190323031832_334329N-ACD-BETA',
  '20190311031858_20190323031858_321021N-ACD-BETA',
  '20190311150811_20190323150811_32612N-ACD-BETA',
  '20190311150837_20190323150837_334221N-ACD-BETA',
  '20190311150901_20190323150902_351341N-ACD-BETA',
  '20190311150926_20190323150926_364317N-ACD-BETA',
  '20190311150951_20190323150951_381253N-ACD-BETA',
  '20190313145101_20190325145102_29398N-ACD-BETA',
  '20190313145127_20190325145127_311036N-ACD-BETA',
  '20190313145152_20190325145152_324151N-ACD-BETA',
  '20190313145217_20190325145217_341141N-ACD-BETA',
  '20190315024548_20190327024548_33342N-ACD-BETA',
  '20190315024609_20190327024609_314015N-ACD-BETA',
  '20190315152323_20190327152323_295711N-ACD-BETA',
  '20190315152402_20190327152402_322515N-ACD-BETA',
  '20190315152427_20190327152428_34122N-ACD-BETA',
  '20190316023903_20190328023903_301317N-ACD-BETA',
  '20190316151553_20190328151554_301260N-ACD-BETA',
  '20190316151619_20190328151619_314912N-ACD-BETA',
  '20190316151644_20190328151644_332035N-ACD-BETA',
  '20190316151709_20190328151709_345012N-ACD-BETA',
  '20190316151733_20190328151734_361949N-ACD-BETA',
  '20190317031806_20190329031806_37498N-ACD-BETA',
  '20190317031831_20190329031831_361754N-ACD-BETA',
  '20190317031855_20190329031856_344817N-ACD-BETA',
  '20190317031920_20190329031921_331840N-ACD-BETA',
  '20190318030904_20190330030904_381040N-ACD-BETA',
  '20190318030929_20190330030929_364057N-ACD-BETA',
  '20190318030953_20190330030954_351116N-ACD-BETA',
  '20190318031019_20190330031020_333952N-ACD-BETA',
  '20190318031050_20190330031051_315647N-ACD-BETA',
  '20190319030154_20190331030155_365157N-ACD-BETA',
  '20190319030220_20190331030220_351849N-ACD-BETA',
  '20190320144244_20190401144245_292628N-ACD-BETA',
  '20190320144310_20190401144310_305757N-ACD-BETA',
  '20190321152406_20190402152406_30129N-ACD-BETA',
  '20190321152431_20190402152431_314335N-ACD-BETA',
  '20190321152456_20190402152456_331316N-ACD-BETA',
  '20190321152520_20190402152521_344254N-ACD-BETA',
  '20190322151452_20190403151453_2933N-ACD-BETA',
  '20190322151518_20190403151518_303915N-ACD-BETA',
  '20190322151542_20190403151542_32910N-ACD-BETA',
  '20190322151618_20190403151618_341210N-ACD-BETA',
  '20190323150811_20190404150811_32611N-ACD-BETA',
  '20190323150837_20190404150837_334220N-ACD-BETA',
  '20190323150902_20190404150902_351340N-ACD-BETA',
  '20190323150926_20190404150927_364317N-ACD-BETA',
  '20190323150951_20190404150952_381253N-ACD-BETA',
  '20190324145918_20190405145918_321325N-ACD-BETA',
  '20190324145943_20190405145944_334933N-ACD-BETA',
  '20190324150033_20190405150033_365032N-ACD-BETA',
  '20190324150058_20190405150058_382020N-ACD-BETA',
  '20190327152323_20190408152324_295710N-ACD-BETA',
  '20190327152402_20190408152402_322514N-ACD-BETA',
  '20190327152428_20190408152428_34121N-ACD-BETA',
  '20190329031806_20190410031806_37497N-ACD-BETA',
  '20190329031831_20190410031831_361753N-ACD-BETA',
  '20190329031856_20190410031856_344815N-ACD-BETA',
  '20190329031921_20190410031921_331839N-ACD-BETA',
  '20190330030904_20190411030904_381040N-ACD-BETA',
  '20190330030929_20190411030929_364057N-ACD-BETA',
  '20190330030954_20190411030954_351116N-ACD-BETA',
  '20190330031020_20190411031020_333952N-ACD-BETA',
  '20190330031051_20190411031051_315647N-ACD-BETA'],
 'apr': ['20190320144244_20190401144245_292628N-ACD-BETA',
  '20190320144310_20190401144310_305757N-ACD-BETA',
  '20190321152406_20190402152406_30129N-ACD-BETA',
  '20190321152431_20190402152431_314335N-ACD-BETA',
  '20190321152456_20190402152456_331316N-ACD-BETA',
  '20190321152520_20190402152521_344254N-ACD-BETA',
  '20190322151452_20190403151453_2933N-ACD-BETA',
  '20190322151518_20190403151518_303915N-ACD-BETA',
  '20190322151542_20190403151542_32910N-ACD-BETA',
  '20190322151618_20190403151618_341210N-ACD-BETA',
  '20190323150811_20190404150811_32611N-ACD-BETA',
  '20190323150837_20190404150837_334220N-ACD-BETA',
  '20190323150902_20190404150902_351340N-ACD-BETA',
  '20190323150926_20190404150927_364317N-ACD-BETA',
  '20190323150951_20190404150952_381253N-ACD-BETA',
  '20190324145918_20190405145918_321325N-ACD-BETA',
  '20190324145943_20190405145944_334933N-ACD-BETA',
  '20190324150033_20190405150033_365032N-ACD-BETA',
  '20190324150058_20190405150058_382020N-ACD-BETA',
  '20190327152323_20190408152324_295710N-ACD-BETA',
  '20190327152402_20190408152402_322514N-ACD-BETA',
  '20190327152428_20190408152428_34121N-ACD-BETA',
  '20190329031806_20190410031806_37497N-ACD-BETA',
  '20190329031831_20190410031831_361753N-ACD-BETA',
  '20190329031856_20190410031856_344815N-ACD-BETA',
  '20190329031921_20190410031921_331839N-ACD-BETA',
  '20190330030904_20190411030904_381040N-ACD-BETA',
  '20190330030929_20190411030929_364057N-ACD-BETA',
  '20190330030954_20190411030954_351116N-ACD-BETA',
  '20190330031020_20190411031020_333952N-ACD-BETA',
  '20190330031051_20190411031051_315647N-ACD-BETA',
  '20190405031018_20190417031019_361418N-ACD-BETA',
  '20190405031108_20190417031108_33150N-ACD-BETA',
  '20190408024548_20190420024549_33341N-ACD-BETA',
  '20190408152402_20190420152403_322514N-ACD-BETA',
  '20190408152428_20190420152429_34122N-ACD-BETA',
  '20190409023903_20190421023904_301315N-ACD-BETA',
  '20190409032625_20190421032625_35031N-ACD-BETA',
  '20190409032650_20190421032650_332911N-ACD-BETA',
  '20190409032715_20190421032715_315932N-ACD-BETA',
  '20190410150643_20190422150643_292129N-ACD-BETA',
  '20190410150709_20190422150709_305242N-ACD-BETA',
  '20190410150733_20190422150734_32245N-ACD-BETA',
  '20190410150825_20190422150825_352653N-ACD-BETA',
  '20190411030904_20190423030905_381040N-ACD-BETA',
  '20190411030929_20190423030930_364057N-ACD-BETA',
  '20190411030954_20190423030955_351117N-ACD-BETA',
  '20190411031020_20190423031020_333953N-ACD-BETA',
  '20190411031051_20190423031052_315648N-ACD-BETA',
  '20190413025305_20190425025306_363435N-ACD-BETA',
  '20190413025331_20190425025331_345832N-ACD-BETA',
  '20190413025356_20190425025356_33275N-ACD-BETA',
  '20190413025428_20190425025428_314348N-ACD-BETA',
  '20190415032800_20190427032801_314616N-ACD-BETA',
  '20190416031717_20190428031718_381412N-ACD-BETA',
  '20190416031742_20190428031743_364429N-ACD-BETA',
  '20190416031807_20190428031808_351452N-ACD-BETA',
  '20190416031833_20190428031833_334330N-ACD-BETA',
  '20190416031858_20190428031859_321022N-ACD-BETA',
  '20190417031130_20190429031131_315020N-ACD-BETA',
  '20190420152324_20190502152325_295712N-ACD-BETA',
  '20190421023904_20190503023904_301317N-ACD-BETA',
  '20190421151554_20190503151555_30130N-ACD-BETA',
  '20190421151620_20190503151621_314913N-ACD-BETA',
  '20190421151645_20190503151645_332035N-ACD-BETA',
  '20190421151710_20190503151710_345012N-ACD-BETA',
  '20190421151735_20190503151735_361950N-ACD-BETA',
  '20190422031921_20190504031922_331841N-ACD-BETA',
  '20190422150734_20190504150734_32247N-ACD-BETA',
  '20190422150759_20190504150759_335347N-ACD-BETA',
  '20190422150825_20190504150826_352655N-ACD-BETA',
  '20190424030155_20190506030156_365158N-ACD-BETA',
  '20190426024632_20190508024632_33017N-ACD-BETA',
  '20190426024657_20190508024657_313022N-ACD-BETA',
  '20190426024721_20190508024721_30037N-ACD-BETA',
  '20190426024744_20190508024745_28347N-ACD-BETA'],
 'may': ['20190420152324_20190502152325_295712N-ACD-BETA',
  '20190421023904_20190503023904_301317N-ACD-BETA',
  '20190421151554_20190503151555_30130N-ACD-BETA',
  '20190421151620_20190503151621_314913N-ACD-BETA',
  '20190421151645_20190503151645_332035N-ACD-BETA',
  '20190421151710_20190503151710_345012N-ACD-BETA',
  '20190421151735_20190503151735_361950N-ACD-BETA',
  '20190422031921_20190504031922_331841N-ACD-BETA',
  '20190422150734_20190504150734_32247N-ACD-BETA',
  '20190422150759_20190504150759_335347N-ACD-BETA',
  '20190422150825_20190504150826_352655N-ACD-BETA',
  '20190424030155_20190506030156_365158N-ACD-BETA',
  '20190426024632_20190508024632_33017N-ACD-BETA',
  '20190426024657_20190508024657_313022N-ACD-BETA',
  '20190426024721_20190508024721_30037N-ACD-BETA',
  '20190426024744_20190508024745_28347N-ACD-BETA',
  '20190502152325_20190514152325_295710N-ACD-BETA',
  '20190502152404_20190514152404_322517N-ACD-BETA',
  '20190502152429_20190514152430_34125N-ACD-BETA',
  '20190502152454_20190514152454_353243N-ACD-BETA',
  '20190503151555_20190515151555_301259N-ACD-BETA',
  '20190503151621_20190515151621_314912N-ACD-BETA',
  '20190503151645_20190515151646_332034N-ACD-BETA',
  '20190503151710_20190515151711_345011N-ACD-BETA',
  '20190503151735_20190515151736_361949N-ACD-BETA',
  '20190504150644_20190516150644_292128N-ACD-BETA',
  '20190504150710_20190516150710_305241N-ACD-BETA',
  '20190504150734_20190516150735_32245N-ACD-BETA',
  '20190504150759_20190516150800_335347N-ACD-BETA',
  '20190504150826_20190516150827_352654N-ACD-BETA',
  '20190504150902_20190516150902_373459N-ACD-BETA',
  '20190505030905_20190517030906_381042N-ACD-BETA',
  '20190505030930_20190517030931_364058N-ACD-BETA',
  '20190505030955_20190517030956_351118N-ACD-BETA',
  '20190505031021_20190517031021_333954N-ACD-BETA',
  '20190505145907_20190517145908_29354N-ACD-BETA',
  '20190505145932_20190517145933_303521N-ACD-BETA',
  '20190505145957_20190517145958_3253N-ACD-BETA',
  '20190505150022_20190517150023_333444N-ACD-BETA',
  '20190505150047_20190517150047_35424N-ACD-BETA',
  '20190505150112_20190517150112_36343N-ACD-BETA',
  '20190505150137_20190517150138_38535N-ACD-BETA',
  '20190506030156_20190518030156_365159N-ACD-BETA',
  '20190506030221_20190518030222_351852N-ACD-BETA',
  '20190506145109_20190518145110_322655N-ACD-BETA',
  '20190506145135_20190518145135_34312N-ACD-BETA',
  '20190506145200_20190518145200_353431N-ACD-BETA',
  '20190506145215_20190518145216_364746N-ACD-BETA',
  '20190507025306_20190519025307_363438N-ACD-BETA',
  '20190507025334_20190519025334_34555N-ACD-BETA',
  '20190507144311_20190519144312_305756N-ACD-BETA',
  '20190509151454_20190521151455_2933N-ACD-BETA',
  '20190509151520_20190521151520_303915N-ACD-BETA',
  '20190509151544_20190521151544_32911N-ACD-BETA',
  '20190509151619_20190521151620_341214N-ACD-BETA',
  '20190509151645_20190521151646_354818N-ACD-BETA',
  '20190509151710_20190521151711_371937N-ACD-BETA',
  '20190510150813_20190522150813_32612N-ACD-BETA',
  '20190510150838_20190522150839_334221N-ACD-BETA',
  '20190510150903_20190522150904_351342N-ACD-BETA',
  '20190510150928_20190522150929_364318N-ACD-BETA',
  '20190510150953_20190522150954_381255N-ACD-BETA',
  '20190511145919_20190523145920_321325N-ACD-BETA',
  '20190511145945_20190523145946_334934N-ACD-BETA',
  '20190511150010_20190523150011_352056N-ACD-BETA',
  '20190511150035_20190523150035_365034N-ACD-BETA',
  '20190511150059_20190523150100_382021N-ACD-BETA',
  '20190512145103_20190524145104_29398N-ACD-BETA',
  '20190514152325_20190526152326_295710N-ACD-BETA',
  '20190514152404_20190526152405_322517N-ACD-BETA',
  '20190515032626_20190527032627_35033N-ACD-BETA',
  '20190515151555_20190527151556_30130N-ACD-BETA',
  '20190515151621_20190527151622_314913N-ACD-BETA',
  '20190515151646_20190527151646_332036N-ACD-BETA',
  '20190515151711_20190527151711_345013N-ACD-BETA',
  '20190515151736_20190527151736_361951N-ACD-BETA',
  '20190516150644_20190528150645_292129N-ACD-BETA',
  '20190516150710_20190528150711_305242N-ACD-BETA',
  '20190516150735_20190528150736_32246N-ACD-BETA',
  '20190516150800_20190528150800_335346N-ACD-BETA',
  '20190516150827_20190528150827_352654N-ACD-BETA',
  '20190516150902_20190528150903_373460N-ACD-BETA',
  '20190517030906_20190529030907_381042N-ACD-BETA',
  '20190517030931_20190529030931_364058N-ACD-BETA',
  '20190517030956_20190529030956_351118N-ACD-BETA',
  '20190517031021_20190529031022_333954N-ACD-BETA',
  '20190517031053_20190529031053_315649N-ACD-BETA',
  '20190517145958_20190529145958_3254N-ACD-BETA',
  '20190517150023_20190529150023_333444N-ACD-BETA',
  '20190517150047_20190529150048_35424N-ACD-BETA',
  '20190517150112_20190529150113_36342N-ACD-BETA',
  '20190517150138_20190529150138_38534N-ACD-BETA',
  '20190518145110_20190530145110_322654N-ACD-BETA',
  '20190518145135_20190530145136_34312N-ACD-BETA',
  '20190518145200_20190530145201_353431N-ACD-BETA',
  '20190518145216_20190530145217_364746N-ACD-BETA',
  '20190519144247_20190531144247_292627N-ACD-BETA',
  '20190520024633_20190601024633_33017N-ACD-BETA',
  '20190520024658_20190601024658_313021N-ACD-BETA',
  '20190520024722_20190601024723_30037N-ACD-BETA',
  '20190520024745_20190601024746_28346N-ACD-BETA',
  '20190520152408_20190601152409_301210N-ACD-BETA',
  '20190520152433_20190601152434_314336N-ACD-BETA',
  '20190520152458_20190601152458_331317N-ACD-BETA',
  '20190520152523_20190601152523_344256N-ACD-BETA',
  '20190521032712_20190602032713_344535N-ACD-BETA',
  '20190521032737_20190602032738_331558N-ACD-BETA',
  '20190521032802_20190602032802_314617N-ACD-BETA',
  '20190521151455_20190602151455_2932N-ACD-BETA',
  '20190521151520_20190602151521_303915N-ACD-BETA',
  '20190521151544_20190602151545_32911N-ACD-BETA',
  '20190521151620_20190602151621_341213N-ACD-BETA',
  '20190521151646_20190602151646_354818N-ACD-BETA',
  '20190521151711_20190602151711_371938N-ACD-BETA',
  '20190522031719_20190603031720_381413N-ACD-BETA',
  '20190522031744_20190603031744_364429N-ACD-BETA',
  '20190522031809_20190603031809_351452N-ACD-BETA',
  '20190522031834_20190603031835_334330N-ACD-BETA',
  '20190522031900_20190603031901_321022N-ACD-BETA',
  '20190522150813_20190603150814_32612N-ACD-BETA',
  '20190522150839_20190603150840_334221N-ACD-BETA',
  '20190522150904_20190603150904_351342N-ACD-BETA',
  '20190522150929_20190603150929_364319N-ACD-BETA',
  '20190522150954_20190603150954_381256N-ACD-BETA',
  '20190523030955_20190604030956_374541N-ACD-BETA',
  '20190523031020_20190604031021_361420N-ACD-BETA',
  '20190523031045_20190604031046_344441N-ACD-BETA',
  '20190523031110_20190604031110_33151N-ACD-BETA',
  '20190523031132_20190604031133_315020N-ACD-BETA',
  '20190523145920_20190604145921_321325N-ACD-BETA',
  '20190523145946_20190604145946_334934N-ACD-BETA',
  '20190523150011_20190604150011_352056N-ACD-BETA',
  '20190523150035_20190604150036_365034N-ACD-BETA',
  '20190523150100_20190604150101_382021N-ACD-BETA',
  '20190524030116_20190605030116_365151N-ACD-BETA',
  '20190524030142_20190605030142_351543N-ACD-BETA',
  '20190524030206_20190605030207_334416N-ACD-BETA',
  '20190524030239_20190605030239_3217N-ACD-BETA',
  '20190524145104_20190605145105_29396N-ACD-BETA',
  '20190524145130_20190605145130_311033N-ACD-BETA',
  '20190524145155_20190605145155_324149N-ACD-BETA',
  '20190526152455_20190607152456_353244N-ACD-BETA',
  '20190527151646_20190608151647_332036N-ACD-BETA',
  '20190527151711_20190608151712_345012N-ACD-BETA',
  '20190528150645_20190609150646_292129N-ACD-BETA',
  '20190528150711_20190609150711_305242N-ACD-BETA',
  '20190528150736_20190609150736_32246N-ACD-BETA',
  '20190528150800_20190609150801_335346N-ACD-BETA',
  '20190528150827_20190609150828_352653N-ACD-BETA',
  '20190528150903_20190609150904_373459N-ACD-BETA',
  '20190529145909_20190610145909_29357N-ACD-BETA',
  '20190529145933_20190610145934_303524N-ACD-BETA',
  '20190529145958_20190610145959_3255N-ACD-BETA',
  '20190529150023_20190610150024_333445N-ACD-BETA',
  '20190529150138_20190610150139_38536N-ACD-BETA',
  '20190530030223_20190611030223_351851N-ACD-BETA',
  '20190530145110_20190611145111_322654N-ACD-BETA',
  '20190530145136_20190611145137_34312N-ACD-BETA',
  '20190530145201_20190611145202_353431N-ACD-BETA',
  '20190530145217_20190611145217_364746N-ACD-BETA'],
 'jun': ['20190520024633_20190601024633_33017N-ACD-BETA',
  '20190520024658_20190601024658_313021N-ACD-BETA',
  '20190520024722_20190601024723_30037N-ACD-BETA',
  '20190520024745_20190601024746_28346N-ACD-BETA',
  '20190520152408_20190601152409_301210N-ACD-BETA',
  '20190520152433_20190601152434_314336N-ACD-BETA',
  '20190520152458_20190601152458_331317N-ACD-BETA',
  '20190520152523_20190601152523_344256N-ACD-BETA',
  '20190521032712_20190602032713_344535N-ACD-BETA',
  '20190521032737_20190602032738_331558N-ACD-BETA',
  '20190521032802_20190602032802_314617N-ACD-BETA',
  '20190521151455_20190602151455_2932N-ACD-BETA',
  '20190521151520_20190602151521_303915N-ACD-BETA',
  '20190521151544_20190602151545_32911N-ACD-BETA',
  '20190521151620_20190602151621_341213N-ACD-BETA',
  '20190521151646_20190602151646_354818N-ACD-BETA',
  '20190521151711_20190602151711_371938N-ACD-BETA',
  '20190522031719_20190603031720_381413N-ACD-BETA',
  '20190522031744_20190603031744_364429N-ACD-BETA',
  '20190522031809_20190603031809_351452N-ACD-BETA',
  '20190522031834_20190603031835_334330N-ACD-BETA',
  '20190522031900_20190603031901_321022N-ACD-BETA',
  '20190522150813_20190603150814_32612N-ACD-BETA',
  '20190522150839_20190603150840_334221N-ACD-BETA',
  '20190522150904_20190603150904_351342N-ACD-BETA',
  '20190522150929_20190603150929_364319N-ACD-BETA',
  '20190522150954_20190603150954_381256N-ACD-BETA',
  '20190523030955_20190604030956_374541N-ACD-BETA',
  '20190523031020_20190604031021_361420N-ACD-BETA',
  '20190523031045_20190604031046_344441N-ACD-BETA',
  '20190523031110_20190604031110_33151N-ACD-BETA',
  '20190523031132_20190604031133_315020N-ACD-BETA',
  '20190523145920_20190604145921_321325N-ACD-BETA',
  '20190523145946_20190604145946_334934N-ACD-BETA',
  '20190523150011_20190604150011_352056N-ACD-BETA',
  '20190523150035_20190604150036_365034N-ACD-BETA',
  '20190523150100_20190604150101_382021N-ACD-BETA',
  '20190524030116_20190605030116_365151N-ACD-BETA',
  '20190524030142_20190605030142_351543N-ACD-BETA',
  '20190524030206_20190605030207_334416N-ACD-BETA',
  '20190524030239_20190605030239_3217N-ACD-BETA',
  '20190524145104_20190605145105_29396N-ACD-BETA',
  '20190524145130_20190605145130_311033N-ACD-BETA',
  '20190524145155_20190605145155_324149N-ACD-BETA',
  '20190526152455_20190607152456_353244N-ACD-BETA',
  '20190527151646_20190608151647_332036N-ACD-BETA',
  '20190527151711_20190608151712_345012N-ACD-BETA',
  '20190528150645_20190609150646_292129N-ACD-BETA',
  '20190528150711_20190609150711_305242N-ACD-BETA',
  '20190528150736_20190609150736_32246N-ACD-BETA',
  '20190528150800_20190609150801_335346N-ACD-BETA',
  '20190528150827_20190609150828_352653N-ACD-BETA',
  '20190528150903_20190609150904_373459N-ACD-BETA',
  '20190529145909_20190610145909_29357N-ACD-BETA',
  '20190529145933_20190610145934_303524N-ACD-BETA',
  '20190529145958_20190610145959_3255N-ACD-BETA',
  '20190529150023_20190610150024_333445N-ACD-BETA',
  '20190529150138_20190610150139_38536N-ACD-BETA',
  '20190530030223_20190611030223_351851N-ACD-BETA',
  '20190530145110_20190611145111_322654N-ACD-BETA',
  '20190530145136_20190611145137_34312N-ACD-BETA',
  '20190530145201_20190611145202_353431N-ACD-BETA',
  '20190530145217_20190611145217_364746N-ACD-BETA',
  '20190602151455_20190614151456_2933N-ACD-BETA',
  '20190602151521_20190614151522_303915N-ACD-BETA',
  '20190602151545_20190614151546_32912N-ACD-BETA',
  '20190602151621_20190614151622_341213N-ACD-BETA',
  '20190603031720_20190615031720_381413N-ACD-BETA',
  '20190603031744_20190615031745_364429N-ACD-BETA',
  '20190603031809_20190615031810_351452N-ACD-BETA',
  '20190603031835_20190615031836_334330N-ACD-BETA',
  '20190603031901_20190615031901_321022N-ACD-BETA',
  '20190605145220_20190617145221_341137N-ACD-BETA',
  '20190606025357_20190618025357_3637N-ACD-BETA',
  '20190606025421_20190618025422_343331N-ACD-BETA',
  '20190609031809_20190621031810_37498N-ACD-BETA',
  '20190609031834_20190621031834_361754N-ACD-BETA',
  '20190609031859_20190621031859_344817N-ACD-BETA',
  '20190609031923_20190621031924_331841N-ACD-BETA',
  '20190611030157_20190623030158_365158N-ACD-BETA',
  '20190611030223_20190623030224_351850N-ACD-BETA',
  '20190612144247_20190624144248_292627N-ACD-BETA',
  '20190612144313_20190624144314_305757N-ACD-BETA',
  '20190613152434_20190625152435_314336N-ACD-BETA',
  '20190613152459_20190625152500_331317N-ACD-BETA',
  '20190613152524_20190625152525_344255N-ACD-BETA',
  '20190616031133_20190628031134_315021N-ACD-BETA',
  '20190616145921_20190604145921_321325N-ACD-BETA',
  '20190616145947_20190604145946_334934N-ACD-BETA',
  '20190616150012_20190604150011_352056N-ACD-BETA',
  '20190616150037_20190604150036_365034N-ACD-BETA',
  '20190616150102_20190604150101_382022N-ACD-BETA',
  '20190617030117_20190605030116_365150N-ACD-BETA',
  '20190617145131_20190629145132_311033N-ACD-BETA',
  '20190618025448_20190630025449_33160N-ACD-BETA',
  '20190619152327_20190607152327_295710N-ACD-BETA',
  '20190619152406_20190607152405_322518N-ACD-BETA',
  '20190619152432_20190607152431_34126N-ACD-BETA',
  '20190619152456_20190607152456_353245N-ACD-BETA',
  '20190621150646_20190609150646_292131N-ACD-BETA',
  '20190621150712_20190609150711_305243N-ACD-BETA',
  '20190621150737_20190609150736_32246N-ACD-BETA',
  '20190621150802_20190609150801_335346N-ACD-BETA',
  '20190621150829_20190609150828_352654N-ACD-BETA',
  '20190621150904_20190609150904_373458N-ACD-BETA',
  '20190622030908_20190610030907_381042N-ACD-BETA',
  '20190622030933_20190610030932_364058N-ACD-BETA',
  '20190622030958_20190610030957_351117N-ACD-BETA',
  '20190622031023_20190610031023_333953N-ACD-BETA',
  '20190622031055_20190610031054_315647N-ACD-BETA',
  '20190623145112_20190611145111_322655N-ACD-BETA',
  '20190623145137_20190611145137_34312N-ACD-BETA',
  '20190623145202_20190611145202_353431N-ACD-BETA',
  '20190623145218_20190611145217_364746N-ACD-BETA',
  '20190625024747_20190707024748_28346N-ACD-BETA',
  '20190625152410_20190707152411_301211N-ACD-BETA',
  '20190626032714_20190708032715_344539N-ACD-BETA',
  '20190626032739_20190708032740_331558N-ACD-BETA',
  '20190626032804_20190708032804_314616N-ACD-BETA',
  '20190627031810_20190615031810_351453N-ACD-BETA',
  '20190627150815_20190709150816_32611N-ACD-BETA',
  '20190627150841_20190709150842_334221N-ACD-BETA',
  '20190628030957_20190710030958_374537N-ACD-BETA',
  '20190628031022_20190710031023_361421N-ACD-BETA',
  '20190628031047_20190710031048_344442N-ACD-BETA',
  '20190628031112_20190710031113_33154N-ACD-BETA',
  '20190628031134_20190710031135_315020N-ACD-BETA',
  '20190628145922_20190616145921_321325N-ACD-BETA',
  '20190628145922_20190710145923_321325N-ACD-BETA',
  '20190628145948_20190710145948_334941N-ACD-BETA',
  '20190629145106_20190711145107_29395N-ACD-BETA',
  '20190629145132_20190711145132_311034N-ACD-BETA',
  '20190629145157_20190711145157_324157N-ACD-BETA',
  '20190629145221_20190711145222_341143N-ACD-BETA',
  '20190629145246_20190711145247_354120N-ACD-BETA',
  '20190629145311_20190711145312_371054N-ACD-BETA',
  '20190701152407_20190619152406_322518N-ACD-BETA',
  '20190702032654_20190620032653_332912N-ACD-BETA',
  '20190702032719_20190620032718_315932N-ACD-BETA',
  '20190706025337_20190624025336_34554N-ACD-BETA'],
 'jul': ['20190625024747_20190707024748_28346N-ACD-BETA',
  '20190625152410_20190707152411_301211N-ACD-BETA',
  '20190626032714_20190708032715_344539N-ACD-BETA',
  '20190626032739_20190708032740_331558N-ACD-BETA',
  '20190626032804_20190708032804_314616N-ACD-BETA',
  '20190627150815_20190709150816_32611N-ACD-BETA',
  '20190627150841_20190709150842_334221N-ACD-BETA',
  '20190628030957_20190710030958_374537N-ACD-BETA',
  '20190628031022_20190710031023_361421N-ACD-BETA',
  '20190628031047_20190710031048_344442N-ACD-BETA',
  '20190628031112_20190710031113_33154N-ACD-BETA',
  '20190628031134_20190710031135_315020N-ACD-BETA',
  '20190628145922_20190710145923_321325N-ACD-BETA',
  '20190628145948_20190710145948_334941N-ACD-BETA',
  '20190629145106_20190711145107_29395N-ACD-BETA',
  '20190629145132_20190711145132_311034N-ACD-BETA',
  '20190629145157_20190711145157_324157N-ACD-BETA',
  '20190629145221_20190711145222_341143N-ACD-BETA',
  '20190629145246_20190711145247_354120N-ACD-BETA',
  '20190629145311_20190711145312_371054N-ACD-BETA',
  '20190701152407_20190619152406_322518N-ACD-BETA',
  '20190701152407_20190713152407_322518N-ACD-BETA',
  '20190701152432_20190713152433_34124N-ACD-BETA',
  '20190702032654_20190620032653_332912N-ACD-BETA',
  '20190702032719_20190620032718_315932N-ACD-BETA',
  '20190702032719_20190714032719_315932N-ACD-BETA',
  '20190702151558_20190714151559_301260N-ACD-BETA',
  '20190702151624_20190714151624_314911N-ACD-BETA',
  '20190703031810_20190715031811_37494N-ACD-BETA',
  '20190703031835_20190715031836_361755N-ACD-BETA',
  '20190703031900_20190715031901_344817N-ACD-BETA',
  '20190703031925_20190715031926_331840N-ACD-BETA',
  '20190703150647_20190715150648_292128N-ACD-BETA',
  '20190703150713_20190715150714_305241N-ACD-BETA',
  '20190703150905_20190715150906_37359N-ACD-BETA',
  '20190704030958_20190716030959_351118N-ACD-BETA',
  '20190704031024_20190716031025_333956N-ACD-BETA',
  '20190704031055_20190716031056_315647N-ACD-BETA',
  '20190705030159_20190717030200_365159N-ACD-BETA',
  '20190705030225_20190717030226_351852N-ACD-BETA',
  '20190706025337_20190624025336_34554N-ACD-BETA',
  '20190706144249_20190718144250_292629N-ACD-BETA',
  '20190706144315_20190718144316_305812N-ACD-BETA',
  '20190707024635_20190719024636_3309N-ACD-BETA',
  '20190707024700_20190719024701_313021N-ACD-BETA',
  '20190707024725_20190719024726_30037N-ACD-BETA',
  '20190707024748_20190719024749_28345N-ACD-BETA',
  '20190707152411_20190719152412_301210N-ACD-BETA',
  '20190708032740_20190720032740_331559N-ACD-BETA',
  '20190708032804_20190720032805_314616N-ACD-BETA',
  '20190709031812_20190721031812_351453N-ACD-BETA',
  '20190709031837_20190721031838_334331N-ACD-BETA',
  '20190709031903_20190721031904_321022N-ACD-BETA',
  '20190709150842_20190721150843_334222N-ACD-BETA',
  '20190710030958_20190722030959_374537N-ACD-BETA',
  '20190710031023_20190722031024_361422N-ACD-BETA',
  '20190710031048_20190722031049_344443N-ACD-BETA',
  '20190710031113_20190722031113_33155N-ACD-BETA',
  '20190710031135_20190722031135_315022N-ACD-BETA',
  '20190710145923_20190722145924_321326N-ACD-BETA',
  '20190710145948_20190722145949_334942N-ACD-BETA',
  '20190712025423_20190724025424_343326N-ACD-BETA',
  '20190712025449_20190724025450_3322N-ACD-BETA',
  '20190714023908_20190726023909_301313N-ACD-BETA',
  '20190714032630_20190726032631_35034N-ACD-BETA',
  '20190714032655_20190726032655_332911N-ACD-BETA',
  '20190714032719_20190726032720_315931N-ACD-BETA',
  '20190715031811_20190727031812_37495N-ACD-BETA',
  '20190715031836_20190727031837_361756N-ACD-BETA',
  '20190715031901_20190727031902_344818N-ACD-BETA',
  '20190715031926_20190727031926_331841N-ACD-BETA',
  '20190716030959_20190728031000_351117N-ACD-BETA',
  '20190716031025_20190728031026_333955N-ACD-BETA',
  '20190716031056_20190728031057_315647N-ACD-BETA',
  '20190717030200_20190729030200_365158N-ACD-BETA',
  '20190717030226_20190729030226_351851N-ACD-BETA',
  '20190717145139_20190729145140_34317N-ACD-BETA',
  '20190718144250_20190730144251_292630N-ACD-BETA',
  '20190718144316_20190730144316_305812N-ACD-BETA',
  '20190719024636_20190731024637_3309N-ACD-BETA',
  '20190719024701_20190731024702_313021N-ACD-BETA',
  '20190719024726_20190731024726_30037N-ACD-BETA',
  '20190719024749_20190731024749_28346N-ACD-BETA',
  '20190719152412_20190731152412_301211N-ACD-BETA',
  '20190720032716_20190801032716_344539N-ACD-BETA',
  '20190720032740_20190801032741_331558N-ACD-BETA',
  '20190720032805_20190801032806_314617N-ACD-BETA',
  '20190721031812_20190802031813_351453N-ACD-BETA',
  '20190721031838_20190802031838_334331N-ACD-BETA',
  '20190721031904_20190802031904_321021N-ACD-BETA',
  '20190722030959_20190803031000_374537N-ACD-BETA',
  '20190722031024_20190803031024_361420N-ACD-BETA',
  '20190722031049_20190803031049_344442N-ACD-BETA',
  '20190722031113_20190803031114_33154N-ACD-BETA',
  '20190722031135_20190803031136_315021N-ACD-BETA',
  '20190723030119_20190804030120_365148N-ACD-BETA',
  '20190723030145_20190804030146_351536N-ACD-BETA',
  '20190723145107_20190804145108_29394N-ACD-BETA',
  '20190723145133_20190804145134_311033N-ACD-BETA',
  '20190723145158_20190804145159_324156N-ACD-BETA',
  '20190723145223_20190804145224_341143N-ACD-BETA',
  '20190723145248_20190804145249_354120N-ACD-BETA',
  '20190723145313_20190804145313_371053N-ACD-BETA',
  '20190724025424_20190805025425_343324N-ACD-BETA',
  '20190724025450_20190805025451_3321N-ACD-BETA',
  '20190725152408_20190806152409_322517N-ACD-BETA',
  '20190725152434_20190806152435_34123N-ACD-BETA',
  '20190726023909_20190807023910_301312N-ACD-BETA',
  '20190726032631_20190807032631_35035N-ACD-BETA',
  '20190726032655_20190807032656_332911N-ACD-BETA',
  '20190726032720_20190807032721_315930N-ACD-BETA',
  '20190726151600_20190807151600_301259N-ACD-BETA',
  '20190726151625_20190807151626_314910N-ACD-BETA',
  '20190727150649_20190808150649_292127N-ACD-BETA',
  '20190727150714_20190808150715_305240N-ACD-BETA',
  '20190727150907_20190808150907_37359N-ACD-BETA',
  '20190728031000_20190809031001_351117N-ACD-BETA',
  '20190728031026_20190809031026_333955N-ACD-BETA',
  '20190728031057_20190809031058_315648N-ACD-BETA',
  '20190729030200_20190810030201_365158N-ACD-BETA',
  '20190729030226_20190810030227_351850N-ACD-BETA',
  '20190729145140_20190810145140_34316N-ACD-BETA',
  '20190731024637_20190812024638_3309N-ACD-BETA',
  '20190731024702_20190812024703_313020N-ACD-BETA',
  '20190731024726_20190812024727_30036N-ACD-BETA',
  '20190731024749_20190812024750_28345N-ACD-BETA'],
 'aug': ['20190720032716_20190801032716_344539N-ACD-BETA',
  '20190720032740_20190801032741_331558N-ACD-BETA',
  '20190720032805_20190801032806_314617N-ACD-BETA',
  '20190721031812_20190802031813_351453N-ACD-BETA',
  '20190721031838_20190802031838_334331N-ACD-BETA',
  '20190721031904_20190802031904_321021N-ACD-BETA',
  '20190722030959_20190803031000_374537N-ACD-BETA',
  '20190722031024_20190803031024_361420N-ACD-BETA',
  '20190722031049_20190803031049_344442N-ACD-BETA',
  '20190722031113_20190803031114_33154N-ACD-BETA',
  '20190722031135_20190803031136_315021N-ACD-BETA',
  '20190723030119_20190804030120_365148N-ACD-BETA',
  '20190723030145_20190804030146_351536N-ACD-BETA',
  '20190723145107_20190804145108_29394N-ACD-BETA',
  '20190723145133_20190804145134_311033N-ACD-BETA',
  '20190723145158_20190804145159_324156N-ACD-BETA',
  '20190723145223_20190804145224_341143N-ACD-BETA',
  '20190723145248_20190804145249_354120N-ACD-BETA',
  '20190723145313_20190804145313_371053N-ACD-BETA',
  '20190724025424_20190805025425_343324N-ACD-BETA',
  '20190724025450_20190805025451_3321N-ACD-BETA',
  '20190725152408_20190806152409_322517N-ACD-BETA',
  '20190725152434_20190806152435_34123N-ACD-BETA',
  '20190726023909_20190807023910_301312N-ACD-BETA',
  '20190726032631_20190807032631_35035N-ACD-BETA',
  '20190726032655_20190807032656_332911N-ACD-BETA',
  '20190726032720_20190807032721_315930N-ACD-BETA',
  '20190726151600_20190807151600_301259N-ACD-BETA',
  '20190726151625_20190807151626_314910N-ACD-BETA',
  '20190727150649_20190808150649_292127N-ACD-BETA',
  '20190727150714_20190808150715_305240N-ACD-BETA',
  '20190727150907_20190808150907_37359N-ACD-BETA',
  '20190728031000_20190809031001_351117N-ACD-BETA',
  '20190728031026_20190809031026_333955N-ACD-BETA',
  '20190728031057_20190809031058_315648N-ACD-BETA',
  '20190729030200_20190810030201_365158N-ACD-BETA',
  '20190729030226_20190810030227_351850N-ACD-BETA',
  '20190729145140_20190810145140_34316N-ACD-BETA',
  '20190731024637_20190812024638_3309N-ACD-BETA',
  '20190731024702_20190812024703_313020N-ACD-BETA',
  '20190731024726_20190812024727_30036N-ACD-BETA',
  '20190731024749_20190812024750_28345N-ACD-BETA',
  '20190801032716_20190813032717_344539N-ACD-BETA',
  '20190801032741_20190813032742_331558N-ACD-BETA',
  '20190801032806_20190813032807_314616N-ACD-BETA',
  '20190802031813_20190814031814_351454N-ACD-BETA',
  '20190802031838_20190814031839_334331N-ACD-BETA',
  '20190802031904_20190814031905_321021N-ACD-BETA',
  '20190803145924_20190815145925_321325N-ACD-BETA',
  '20190803145950_20190815145951_334940N-ACD-BETA',
  '20190804145108_20190816145109_29395N-ACD-BETA',
  '20190804145134_20190816145135_311034N-ACD-BETA',
  '20190804145159_20190816145200_324156N-ACD-BETA',
  '20190804145224_20190816145225_341142N-ACD-BETA',
  '20190804145249_20190816145249_354119N-ACD-BETA',
  '20190804145313_20190816145314_371053N-ACD-BETA',
  '20190807023910_20190819023911_301311N-ACD-BETA',
  '20190807032631_20190819032632_35035N-ACD-BETA',
  '20190807032656_20190819032657_332912N-ACD-BETA',
  '20190807032721_20190819032722_315931N-ACD-BETA',
  '20190808150649_20190820150650_292129N-ACD-BETA',
  '20190808150715_20190820150716_305241N-ACD-BETA',
  '20190808150907_20190820150908_37358N-ACD-BETA',
  '20190809031001_20190821031001_351118N-ACD-BETA',
  '20190809031026_20190821031027_333955N-ACD-BETA',
  '20190809031058_20190821031058_315648N-ACD-BETA',
  '20190809145902_20190821145903_282543N-ACD-BETA',
  '20190809145928_20190821145929_295713N-ACD-BETA',
  '20190809145953_20190821145953_312829N-ACD-BETA',
  '20190809150018_20190821150018_325812N-ACD-BETA',
  '20190809150042_20190821150043_342757N-ACD-BETA',
  '20190809150107_20190821150108_355737N-ACD-BETA',
  '20190809150133_20190821150134_372842N-ACD-BETA',
  '20190810145115_20190822145115_322659N-ACD-BETA',
  '20190810145140_20190822145141_34317N-ACD-BETA',
  '20190812024638_20190824024638_3309N-ACD-BETA',
  '20190812024703_20190824024703_313021N-ACD-BETA',
  '20190812024727_20190824024728_30037N-ACD-BETA',
  '20190812024750_20190824024751_28345N-ACD-BETA',
  '20190812152413_20190824152414_301211N-ACD-BETA',
  '20190813032717_20190825032718_344540N-ACD-BETA',
  '20190813032742_20190825032743_331558N-ACD-BETA',
  '20190813032807_20190825032807_314617N-ACD-BETA',
  '20190813151500_20190825151500_2933N-ACD-BETA',
  '20190813151525_20190825151526_303914N-ACD-BETA',
  '20190813151651_20190825151651_354819N-ACD-BETA',
  '20190814031814_20190826031814_351452N-ACD-BETA',
  '20190814031839_20190826031840_334331N-ACD-BETA',
  '20190814031905_20190826031906_321021N-ACD-BETA',
  '20190814150819_20190826150819_32610N-ACD-BETA',
  '20190814150844_20190826150845_334220N-ACD-BETA',
  '20190815145925_20190827145926_321326N-ACD-BETA',
  '20190816145225_20190828145225_341141N-ACD-BETA',
  '20190817025452_20190829025452_33160N-ACD-BETA',
  '20190819023911_20190831023911_301311N-ACD-BETA',
  '20190819032632_20190831032633_35035N-ACD-BETA',
  '20190819032657_20190831032657_332911N-ACD-BETA',
  '20190819032722_20190831032722_315931N-ACD-BETA',
  '20190821031001_20190902031002_351118N-ACD-BETA',
  '20190821031027_20190902031028_333955N-ACD-BETA',
  '20190821031058_20190902031059_315647N-ACD-BETA',
  '20190822030228_20190903030228_351848N-ACD-BETA',
  '20190822145115_20190903145116_32270N-ACD-BETA',
  '20190822145141_20190903145142_34318N-ACD-BETA',
  '20190823025312_20190904025313_363433N-ACD-BETA',
  '20190823025338_20190904025339_345826N-ACD-BETA',
  '20190823144252_20190904144253_292631N-ACD-BETA',
  '20190823144318_20190904144319_305814N-ACD-BETA',
  '20190824024638_20190905024639_3308N-ACD-BETA',
  '20190824024703_20190905024704_313019N-ACD-BETA',
  '20190824024728_20190905024729_30036N-ACD-BETA',
  '20190824024751_20190905024752_28344N-ACD-BETA',
  '20190824152414_20190905152414_301212N-ACD-BETA',
  '20190825032718_20190906032718_344538N-ACD-BETA',
  '20190825032743_20190906032743_331557N-ACD-BETA',
  '20190825032807_20190906032808_314615N-ACD-BETA',
  '20190825151526_20190906151527_303915N-ACD-BETA',
  '20190825151626_20190906151626_341213N-ACD-BETA',
  '20190825151651_20190906151652_354818N-ACD-BETA',
  '20190826031814_20190907031815_351452N-ACD-BETA',
  '20190826031840_20190907031841_334330N-ACD-BETA',
  '20190826031906_20190907031906_321021N-ACD-BETA',
  '20190826150819_20190907150820_32612N-ACD-BETA',
  '20190826150845_20190907150845_334221N-ACD-BETA',
  '20190827031001_20190908031002_374537N-ACD-BETA',
  '20190827031026_20190908031026_361421N-ACD-BETA',
  '20190827031051_20190908031051_344442N-ACD-BETA',
  '20190827031116_20190908031116_33154N-ACD-BETA',
  '20190827031138_20190908031138_315021N-ACD-BETA',
  '20190829025402_20190910025402_3636N-ACD-BETA',
  '20190829025426_20190910025427_343323N-ACD-BETA',
  '20190829025452_20190910025453_33160N-ACD-BETA',
  '20190831032633_20190912032633_35033N-ACD-BETA',
  '20190831032657_20190912032658_332910N-ACD-BETA',
  '20190831032722_20190912032723_315930N-ACD-BETA',
  '20190831151602_20190912151602_30130N-ACD-BETA',
  '20190831151627_20190912151628_314911N-ACD-BETA'],
 'sep': ['20190821031001_20190902031002_351118N-ACD-BETA',
  '20190821031027_20190902031028_333955N-ACD-BETA',
  '20190821031058_20190902031059_315647N-ACD-BETA',
  '20190822030228_20190903030228_351848N-ACD-BETA',
  '20190822145115_20190903145116_32270N-ACD-BETA',
  '20190822145141_20190903145142_34318N-ACD-BETA',
  '20190823025312_20190904025313_363433N-ACD-BETA',
  '20190823025338_20190904025339_345826N-ACD-BETA',
  '20190823144252_20190904144253_292631N-ACD-BETA',
  '20190823144318_20190904144319_305814N-ACD-BETA',
  '20190824024638_20190905024639_3308N-ACD-BETA',
  '20190824024703_20190905024704_313019N-ACD-BETA',
  '20190824024728_20190905024729_30036N-ACD-BETA',
  '20190824024751_20190905024752_28344N-ACD-BETA',
  '20190824152414_20190905152414_301212N-ACD-BETA',
  '20190825032718_20190906032718_344538N-ACD-BETA',
  '20190825032743_20190906032743_331557N-ACD-BETA',
  '20190825032807_20190906032808_314615N-ACD-BETA',
  '20190825151526_20190906151527_303915N-ACD-BETA',
  '20190825151626_20190906151626_341213N-ACD-BETA',
  '20190825151651_20190906151652_354818N-ACD-BETA',
  '20190826031814_20190907031815_351452N-ACD-BETA',
  '20190826031840_20190907031841_334330N-ACD-BETA',
  '20190826031906_20190907031906_321021N-ACD-BETA',
  '20190826150819_20190907150820_32612N-ACD-BETA',
  '20190826150845_20190907150845_334221N-ACD-BETA',
  '20190827031001_20190908031002_374537N-ACD-BETA',
  '20190827031026_20190908031026_361421N-ACD-BETA',
  '20190827031051_20190908031051_344442N-ACD-BETA',
  '20190827031116_20190908031116_33154N-ACD-BETA',
  '20190827031138_20190908031138_315021N-ACD-BETA',
  '20190829025402_20190910025402_3636N-ACD-BETA',
  '20190829025426_20190910025427_343323N-ACD-BETA',
  '20190829025452_20190910025453_33160N-ACD-BETA',
  '20190831032633_20190912032633_35033N-ACD-BETA',
  '20190831032657_20190912032658_332910N-ACD-BETA',
  '20190831032722_20190912032723_315930N-ACD-BETA',
  '20190831151602_20190912151602_30130N-ACD-BETA',
  '20190831151627_20190912151628_314911N-ACD-BETA',
  '20190901150651_20190913150651_292130N-ACD-BETA',
  '20190901150716_20190913150717_305242N-ACD-BETA',
  '20190901150909_20190913150909_373510N-ACD-BETA',
  '20190902031002_20190914031002_351117N-ACD-BETA',
  '20190902031028_20190914031028_333954N-ACD-BETA',
  '20190902031059_20190914031100_315646N-ACD-BETA',
  '20190902145903_20190914145904_282544N-ACD-BETA',
  '20190902145929_20190914145930_295714N-ACD-BETA',
  '20190902145954_20190914145954_312830N-ACD-BETA',
  '20190902150019_20190914150019_325813N-ACD-BETA',
  '20190903030203_20190915030203_365157N-ACD-BETA',
  '20190903030228_20190915030229_351850N-ACD-BETA',
  '20190903145116_20190915145117_322660N-ACD-BETA',
  '20190903145142_20190915145142_34318N-ACD-BETA',
  '20190904025313_20190916025313_363433N-ACD-BETA',
  '20190904025339_20190916025339_345826N-ACD-BETA',
  '20190905024639_20190917024639_3308N-ACD-BETA',
  '20190905024704_20190917024704_313020N-ACD-BETA',
  '20190905024729_20190917024729_30036N-ACD-BETA',
  '20190905024752_20190917024752_28344N-ACD-BETA',
  '20190906032718_20190918032719_344538N-ACD-BETA',
  '20190906032743_20190918032744_331557N-ACD-BETA',
  '20190906032808_20190918032808_314616N-ACD-BETA',
  '20190907031815_20190919031815_351452N-ACD-BETA',
  '20190908031002_20190920031002_374536N-ACD-BETA',
  '20190908031026_20190920031027_361420N-ACD-BETA',
  '20190908031051_20190920031052_344441N-ACD-BETA',
  '20190908031116_20190920031117_33153N-ACD-BETA',
  '20190908031138_20190920031139_315019N-ACD-BETA',
  '20190909030122_20190921030122_365147N-ACD-BETA',
  '20190909030148_20190921030148_351535N-ACD-BETA',
  '20190909145136_20190921145136_311035N-ACD-BETA',
  '20190909145201_20190921145201_324158N-ACD-BETA',
  '20190909145226_20190921145226_341144N-ACD-BETA',
  '20190909145250_20190921145251_354121N-ACD-BETA',
  '20190909145315_20190921145316_371054N-ACD-BETA',
  '20190910025402_20190922025403_3636N-ACD-BETA',
  '20190910025427_20190922025428_343322N-ACD-BETA',
  '20190910025453_20190922025453_33158N-ACD-BETA',
  '20190911152436_20190923152437_34124N-ACD-BETA',
  '20190913150651_20190925150652_292130N-ACD-BETA',
  '20190913150717_20190925150717_305243N-ACD-BETA',
  '20190913150909_20190925150910_373510N-ACD-BETA',
  '20190918151501_20190930151502_2934N-ACD-BETA',
  '20190918151527_20190930151528_303916N-ACD-BETA',
  '20190918151627_20190930151627_341214N-ACD-BETA',
  '20190918151653_20190930151653_354821N-ACD-BETA',
  '20190920145927_20191002145927_321328N-ACD-BETA',
  '20190920145952_20191002145953_334943N-ACD-BETA',
  '20190922025403_20191004025403_3635N-ACD-BETA',
  '20190922025428_20191004025428_343321N-ACD-BETA',
  '20190922025453_20191004025454_33157N-ACD-BETA',
  '20190923152411_20191005152411_322520N-ACD-BETA',
  '20190923152437_20191005152437_34125N-ACD-BETA',
  '20190924023912_20191006023912_301311N-ACD-BETA',
  '20190924151603_20191006151603_30131N-ACD-BETA',
  '20190924151628_20191006151629_314912N-ACD-BETA',
  '20190925150652_20191007150652_292131N-ACD-BETA',
  '20190925150717_20191007150717_305244N-ACD-BETA',
  '20190925150910_20191007150910_373510N-ACD-BETA',
  '20190926031003_20191008031003_351116N-ACD-BETA',
  '20190926031029_20191008031029_333954N-ACD-BETA',
  '20190926031100_20191008031100_315646N-ACD-BETA',
  '20190926145930_20191008145930_295715N-ACD-BETA',
  '20190926145955_20191008145955_312831N-ACD-BETA',
  '20190926150020_20191008150020_325814N-ACD-BETA',
  '20190926150045_20191008150045_342759N-ACD-BETA',
  '20190926150109_20191008150110_355740N-ACD-BETA',
  '20190926150135_20191008150135_372844N-ACD-BETA',
  '20190928144254_20191010144254_292631N-ACD-BETA',
  '20190928144320_20191010144320_305815N-ACD-BETA'],
 'oct': ['20190920145927_20191002145927_321328N-ACD-BETA',
  '20190920145952_20191002145953_334943N-ACD-BETA',
  '20190922025403_20191004025403_3635N-ACD-BETA',
  '20190922025428_20191004025428_343321N-ACD-BETA',
  '20190922025453_20191004025454_33157N-ACD-BETA',
  '20190923152411_20191005152411_322520N-ACD-BETA',
  '20190923152437_20191005152437_34125N-ACD-BETA',
  '20190924023912_20191006023912_301311N-ACD-BETA',
  '20190924151603_20191006151603_30131N-ACD-BETA',
  '20190924151628_20191006151629_314912N-ACD-BETA',
  '20190925150652_20191007150652_292131N-ACD-BETA',
  '20190925150717_20191007150717_305244N-ACD-BETA',
  '20190925150910_20191007150910_373510N-ACD-BETA',
  '20190926031003_20191008031003_351116N-ACD-BETA',
  '20190926031029_20191008031029_333954N-ACD-BETA',
  '20190926031100_20191008031100_315646N-ACD-BETA',
  '20190926145930_20191008145930_295715N-ACD-BETA',
  '20190926145955_20191008145955_312831N-ACD-BETA',
  '20190926150020_20191008150020_325814N-ACD-BETA',
  '20190926150045_20191008150045_342759N-ACD-BETA',
  '20190926150109_20191008150110_355740N-ACD-BETA',
  '20190926150135_20191008150135_372844N-ACD-BETA',
  '20190928144254_20191010144254_292631N-ACD-BETA',
  '20190928144320_20191010144320_305815N-ACD-BETA',
  '20191005152411_20191017152412_322521N-ACD-BETA',
  '20191005152437_20191017152437_34127N-ACD-BETA',
  '20191006023912_20191018023912_301311N-ACD-BETA',
  '20191006032634_20191018032634_35032N-ACD-BETA',
  '20191006032659_20191018032659_33298N-ACD-BETA',
  '20191006032723_20191018032724_315928N-ACD-BETA',
  '20191007031840_20191019031840_361754N-ACD-BETA',
  '20191007031905_20191019031905_344816N-ACD-BETA',
  '20191007031930_20191019031930_331839N-ACD-BETA',
  '20191007150652_20191019150652_292131N-ACD-BETA',
  '20191007150717_20191019150718_305244N-ACD-BETA',
  '20191007150910_20191019150910_373512N-ACD-BETA',
  '20191008031003_20191020031003_351116N-ACD-BETA',
  '20191008031029_20191020031029_333953N-ACD-BETA',
  '20191008031100_20191020031100_315645N-ACD-BETA',
  '20191008145904_20191020145904_282545N-ACD-BETA',
  '20191008145930_20191020145930_295715N-ACD-BETA',
  '20191008145955_20191020145955_312831N-ACD-BETA',
  '20191008150020_20191020150020_325814N-ACD-BETA',
  '20191008150045_20191020150045_342759N-ACD-BETA',
  '20191008150110_20191020150109_355740N-ACD-BETA',
  '20191008150135_20191020150135_372844N-ACD-BETA',
  '20191010144254_20191022144254_292632N-ACD-BETA',
  '20191010144320_20191022144319_305815N-ACD-BETA',
  '20191011152415_20191023152415_301213N-ACD-BETA',
  '20191013031816_20191025031816_351451N-ACD-BETA',
  '20191013031842_20191025031842_334329N-ACD-BETA',
  '20191013031907_20191025031907_321020N-ACD-BETA',
  '20191013150821_20191025150821_32614N-ACD-BETA',
  '20191013150846_20191025150846_334222N-ACD-BETA',
  '20191014031117_20191026031117_33153N-ACD-BETA',
  '20191014031139_20191026031139_315020N-ACD-BETA',
  '20191014145927_20191026145927_321328N-ACD-BETA',
  '20191014145953_20191026145953_334943N-ACD-BETA',
  '20191015145111_20191027145111_29398N-ACD-BETA',
  '20191015145137_20191027145137_311036N-ACD-BETA',
  '20191015145202_20191027145202_324159N-ACD-BETA',
  '20191015145226_20191027145226_341144N-ACD-BETA',
  '20191015145251_20191027145251_354121N-ACD-BETA',
  '20191015145316_20191027145316_371055N-ACD-BETA',
  '20191016025403_20191028025403_3637N-ACD-BETA',
  '20191016025428_20191028025428_343324N-ACD-BETA',
  '20191016025454_20191028025454_3320N-ACD-BETA',
  '20191018023912_20191030023912_301312N-ACD-BETA',
  '20191018032634_20191030032634_35032N-ACD-BETA',
  '20191018151603_20191030151603_30132N-ACD-BETA',
  '20191018151629_20191030151629_314912N-ACD-BETA'],
 'nov': ['20181101030117_20181113030116_365149N-ACD-BETA',
  '20181101030142_20181113030142_351541N-ACD-BETA',
  '20181101030207_20181113030207_334413N-ACD-BETA',
  '20181101030239_20181113030239_3214N-ACD-BETA',
  '20181101145105_20181113145104_29398N-ACD-BETA',
  '20181101145131_20181113145130_311035N-ACD-BETA',
  '20181101145155_20181113145155_324151N-ACD-BETA',
  '20181101145220_20181113145220_341140N-ACD-BETA',
  '20181101145245_20181113145245_354118N-ACD-BETA',
  '20181101145310_20181113145310_371054N-ACD-BETA',
  '20181102025357_20181114025357_3634N-ACD-BETA',
  '20181102025422_20181114025421_343327N-ACD-BETA',
  '20181102025447_20181114025447_33158N-ACD-BETA',
  '20181103024551_20181115024551_33341N-ACD-BETA',
  '20181103024612_20181115024612_314013N-ACD-BETA',
  '20181104023906_20181116023906_301318N-ACD-BETA',
  '20181104032628_20181116032627_35030N-ACD-BETA',
  '20181104032652_20181116032652_332910N-ACD-BETA',
  '20181104032717_20181116032717_315929N-ACD-BETA',
  '20181105031809_20181117031809_37497N-ACD-BETA',
  '20181105031834_20181117031834_361752N-ACD-BETA',
  '20181105031859_20181117031858_344816N-ACD-BETA',
  '20181105031923_20181117031923_331839N-ACD-BETA',
  '20181106030907_20181118030907_381040N-ACD-BETA',
  '20181106030932_20181118030932_364056N-ACD-BETA',
  '20181106030957_20181118030956_351116N-ACD-BETA',
  '20181106031022_20181118031022_333951N-ACD-BETA',
  '20181106031054_20181118031054_315645N-ACD-BETA',
  '20181107030223_20181119030223_351848N-ACD-BETA',
  '20181108025307_20181120025307_363435N-ACD-BETA',
  '20181108025333_20181120025333_345831N-ACD-BETA',
  '20181108025358_20181120025358_33274N-ACD-BETA',
  '20181108025430_20181120025430_314347N-ACD-BETA',
  '20181108144247_20181120144247_292630N-ACD-BETA',
  '20181108144313_20181120144313_305759N-ACD-BETA',
  '20181109024634_20181121024633_33014N-ACD-BETA',
  '20181109024658_20181121024658_313019N-ACD-BETA',
  '20181109024723_20181121024723_30036N-ACD-BETA',
  '20181109024746_20181121024746_28345N-ACD-BETA',
  '20181109152409_20181121152409_301210N-ACD-BETA',
  '20181109152434_20181121152434_314336N-ACD-BETA',
  '20181109152459_20181121152458_331317N-ACD-BETA',
  '20181109152523_20181121152523_344255N-ACD-BETA',
  '20181110151455_20181122151455_2935N-ACD-BETA',
  '20181110151521_20181122151521_303917N-ACD-BETA',
  '20181110151545_20181122151545_32914N-ACD-BETA',
  '20181110151621_20181122151621_341213N-ACD-BETA',
  '20181110151647_20181122151646_354817N-ACD-BETA',
  '20181111031720_20181123031719_381411N-ACD-BETA',
  '20181111031744_20181123031744_364428N-ACD-BETA',
  '20181111031809_20181123031809_351450N-ACD-BETA',
  '20181111031835_20181123031835_334327N-ACD-BETA',
  '20181111031901_20181123031901_321019N-ACD-BETA',
  '20181112030956_20181124030956_374539N-ACD-BETA',
  '20181112031021_20181124031021_361419N-ACD-BETA',
  '20181112031046_20181124031045_344440N-ACD-BETA',
  '20181112031111_20181124031110_33151N-ACD-BETA',
  '20181112031133_20181124031132_315020N-ACD-BETA',
  '20181113030116_20181125030116_365149N-ACD-BETA',
  '20181113030142_20181125030142_351541N-ACD-BETA',
  '20181113030207_20181125030207_334413N-ACD-BETA',
  '20181113030239_20181125030239_3214N-ACD-BETA',
  '20181114025357_20181126025356_3635N-ACD-BETA',
  '20181114025421_20181126025421_343327N-ACD-BETA',
  '20181114025447_20181126025447_33158N-ACD-BETA',
  '20181115024551_20181127024550_33341N-ACD-BETA',
  '20181115024612_20181127024612_314013N-ACD-BETA',
  '20181116032627_20181128032627_35030N-ACD-BETA',
  '20181116032652_20181128032652_332910N-ACD-BETA',
  '20181116032717_20181128032717_315929N-ACD-BETA',
  '20181117031858_20181129031858_344816N-ACD-BETA',
  '20181117031923_20181129031923_331840N-ACD-BETA',
  '20181117150645_20181129150645_292131N-ACD-BETA',
  '20181117150711_20181129150710_305243N-ACD-BETA',
  '20181117150736_20181129150735_32248N-ACD-BETA',
  '20181117150801_20181129150800_335348N-ACD-BETA',
  '20181117150827_20181129150827_352656N-ACD-BETA',
  '20181117150903_20181129150903_373458N-ACD-BETA',
  '20181118030907_20181130030906_381040N-ACD-BETA',
  '20181118030932_20181130030931_364056N-ACD-BETA',
  '20181118030956_20181130030956_351116N-ACD-BETA',
  '20181118031022_20181130031022_333951N-ACD-BETA',
  '20181118031054_20181130031053_315647N-ACD-BETA',
  '20181118145858_20181130145857_282545N-ACD-BETA',
  '20181118145924_20181130145923_295716N-ACD-BETA',
  '20181118145948_20181130145948_312831N-ACD-BETA',
  '20181118150013_20181130150013_325813N-ACD-BETA',
  '20181118150038_20181130150038_342752N-ACD-BETA',
  '20181118150103_20181130150102_355731N-ACD-BETA',
  '20181118150129_20181130150128_372847N-ACD-BETA',
  '20181119145111_20181201145110_322656N-ACD-BETA',
  '20181119145136_20181201145136_34314N-ACD-BETA',
  '20181119145201_20181201145201_353433N-ACD-BETA',
  '20181119145217_20181201145216_364749N-ACD-BETA',
  '20181120025307_20181202025307_363434N-ACD-BETA',
  '20181120025333_20181202025333_345830N-ACD-BETA',
  '20181120025358_20181202025357_33273N-ACD-BETA',
  '20181120025430_20181202025430_314346N-ACD-BETA',
  '20181120144247_20181202144247_292629N-ACD-BETA',
  '20181120144313_20181202144313_305759N-ACD-BETA',
  '20181121024633_20181203024633_33014N-ACD-BETA',
  '20181121024658_20181203024658_313020N-ACD-BETA',
  '20181121024723_20181203024723_30036N-ACD-BETA',
  '20181121024746_20181203024746_28344N-ACD-BETA',
  '20181121152409_20181203152408_301210N-ACD-BETA',
  '20181121152434_20181203152433_314337N-ACD-BETA',
  '20181122151455_20181204151455_2935N-ACD-BETA',
  '20181122151521_20181204151520_303917N-ACD-BETA',
  '20181122151545_20181204151544_32913N-ACD-BETA',
  '20181122151621_20181204151620_341213N-ACD-BETA',
  '20181122151646_20181204151646_354818N-ACD-BETA',
  '20181123031719_20181205031719_381411N-ACD-BETA',
  '20181123031744_20181205031744_364428N-ACD-BETA',
  '20181123031809_20181205031809_351451N-ACD-BETA',
  '20181123031835_20181205031834_334329N-ACD-BETA',
  '20181123031901_20181205031900_321020N-ACD-BETA',
  '20181125030116_20181207030116_365150N-ACD-BETA',
  '20181125030142_20181207030141_351542N-ACD-BETA',
  '20181125030207_20181207030206_334413N-ACD-BETA',
  '20181125030239_20181207030239_3215N-ACD-BETA',
  '20181125145104_20181207145104_29397N-ACD-BETA',
  '20181125145130_20181207145130_311035N-ACD-BETA',
  '20181125145155_20181207145154_324151N-ACD-BETA',
  '20181125145220_20181207145219_341140N-ACD-BETA',
  '20181125145244_20181207145244_354118N-ACD-BETA',
  '20181125145309_20181207145309_371054N-ACD-BETA',
  '20181126025356_20181208025356_3635N-ACD-BETA',
  '20181127024550_20181209024550_33341N-ACD-BETA',
  '20181127024612_20181209024611_314012N-ACD-BETA',
  '20181127152326_20181209152325_295712N-ACD-BETA',
  '20181127152404_20181209152404_322516N-ACD-BETA',
  '20181127152430_20181209152430_34124N-ACD-BETA',
  '20181128151556_20181210151555_30131N-ACD-BETA',
  '20181128151621_20181210151621_314913N-ACD-BETA',
  '20181128151646_20181210151646_332035N-ACD-BETA',
  '20181128151711_20181210151711_345012N-ACD-BETA',
  '20181128151736_20181210151736_361949N-ACD-BETA',
  '20181129031808_20181211031808_37499N-ACD-BETA',
  '20181129031833_20181211031833_361754N-ACD-BETA',
  '20181129031858_20181211031857_344817N-ACD-BETA',
  '20181129031923_20181211031922_331840N-ACD-BETA',
  '20181129150645_20181211150644_292131N-ACD-BETA',
  '20181129150710_20181211150710_305244N-ACD-BETA',
  '20181129150735_20181211150735_32248N-ACD-BETA',
  '20181129150800_20181211150800_335348N-ACD-BETA',
  '20181129150827_20181211150826_352655N-ACD-BETA',
  '20181129150903_20181211150902_373458N-ACD-BETA',
  '20181130030906_20181212030906_381040N-ACD-BETA',
  '20181130030931_20181212030931_364057N-ACD-BETA',
  '20181130030956_20181212030955_351117N-ACD-BETA',
  '20181130031022_20181212031021_333952N-ACD-BETA',
  '20181130031053_20181212031053_315647N-ACD-BETA'],
 'dec': ['20181119145111_20181201145110_322656N-ACD-BETA',
  '20181119145136_20181201145136_34314N-ACD-BETA',
  '20181119145201_20181201145201_353433N-ACD-BETA',
  '20181119145217_20181201145216_364749N-ACD-BETA',
  '20181120025307_20181202025307_363434N-ACD-BETA',
  '20181120025333_20181202025333_345830N-ACD-BETA',
  '20181120025358_20181202025357_33273N-ACD-BETA',
  '20181120025430_20181202025430_314346N-ACD-BETA',
  '20181120144247_20181202144247_292629N-ACD-BETA',
  '20181120144313_20181202144313_305759N-ACD-BETA',
  '20181121024633_20181203024633_33014N-ACD-BETA',
  '20181121024658_20181203024658_313020N-ACD-BETA',
  '20181121024723_20181203024723_30036N-ACD-BETA',
  '20181121024746_20181203024746_28344N-ACD-BETA',
  '20181121152409_20181203152408_301210N-ACD-BETA',
  '20181121152434_20181203152433_314337N-ACD-BETA',
  '20181122151455_20181204151455_2935N-ACD-BETA',
  '20181122151521_20181204151520_303917N-ACD-BETA',
  '20181122151545_20181204151544_32913N-ACD-BETA',
  '20181122151621_20181204151620_341213N-ACD-BETA',
  '20181122151646_20181204151646_354818N-ACD-BETA',
  '20181123031719_20181205031719_381411N-ACD-BETA',
  '20181123031744_20181205031744_364428N-ACD-BETA',
  '20181123031809_20181205031809_351451N-ACD-BETA',
  '20181123031835_20181205031834_334329N-ACD-BETA',
  '20181123031901_20181205031900_321020N-ACD-BETA',
  '20181125030116_20181207030116_365150N-ACD-BETA',
  '20181125030142_20181207030141_351542N-ACD-BETA',
  '20181125030207_20181207030206_334413N-ACD-BETA',
  '20181125030239_20181207030239_3215N-ACD-BETA',
  '20181125145104_20181207145104_29397N-ACD-BETA',
  '20181125145130_20181207145130_311035N-ACD-BETA',
  '20181125145155_20181207145154_324151N-ACD-BETA',
  '20181125145220_20181207145219_341140N-ACD-BETA',
  '20181125145244_20181207145244_354118N-ACD-BETA',
  '20181125145309_20181207145309_371054N-ACD-BETA',
  '20181126025356_20181208025356_3635N-ACD-BETA',
  '20181127024550_20181209024550_33341N-ACD-BETA',
  '20181127024612_20181209024611_314012N-ACD-BETA',
  '20181127152326_20181209152325_295712N-ACD-BETA',
  '20181127152404_20181209152404_322516N-ACD-BETA',
  '20181127152430_20181209152430_34124N-ACD-BETA',
  '20181128151556_20181210151555_30131N-ACD-BETA',
  '20181128151621_20181210151621_314913N-ACD-BETA',
  '20181128151646_20181210151646_332035N-ACD-BETA',
  '20181128151711_20181210151711_345012N-ACD-BETA',
  '20181128151736_20181210151736_361949N-ACD-BETA',
  '20181129031808_20181211031808_37499N-ACD-BETA',
  '20181129031833_20181211031833_361754N-ACD-BETA',
  '20181129031858_20181211031857_344817N-ACD-BETA',
  '20181129031923_20181211031922_331840N-ACD-BETA',
  '20181129150645_20181211150644_292131N-ACD-BETA',
  '20181129150710_20181211150710_305244N-ACD-BETA',
  '20181129150735_20181211150735_32248N-ACD-BETA',
  '20181129150800_20181211150800_335348N-ACD-BETA',
  '20181129150827_20181211150826_352655N-ACD-BETA',
  '20181129150903_20181211150902_373458N-ACD-BETA',
  '20181130030906_20181212030906_381040N-ACD-BETA',
  '20181130030931_20181212030931_364057N-ACD-BETA',
  '20181130030956_20181212030955_351117N-ACD-BETA',
  '20181130031022_20181212031021_333952N-ACD-BETA',
  '20181130031053_20181212031053_315647N-ACD-BETA',
  '20181201145110_20181213145109_322656N-ACD-BETA',
  '20181201145136_20181213145135_34314N-ACD-BETA',
  '20181201145201_20181213145200_353433N-ACD-BETA',
  '20181201145216_20181213145216_364748N-ACD-BETA',
  '20181202025430_20181214025429_314347N-ACD-BETA',
  '20181202144247_20181214144246_292630N-ACD-BETA',
  '20181202144313_20181214144312_305759N-ACD-BETA',
  '20181203152408_20181215152408_30129N-ACD-BETA',
  '20181203152433_20181215152433_314336N-ACD-BETA',
  '20181203152458_20181215152457_331318N-ACD-BETA',
  '20181203152523_20181215152522_344255N-ACD-BETA',
  '20181204151455_20181216151454_2935N-ACD-BETA',
  '20181204151520_20181216151520_303917N-ACD-BETA',
  '20181204151544_20181216151544_32913N-ACD-BETA',
  '20181204151620_20181216151620_341213N-ACD-BETA',
  '20181204151646_20181216151645_354817N-ACD-BETA',
  '20181205150814_20181217150813_32611N-ACD-BETA',
  '20181205150839_20181217150838_334221N-ACD-BETA',
  '20181205150904_20181217150903_351341N-ACD-BETA',
  '20181205150929_20181217150928_364318N-ACD-BETA',
  '20181205150954_20181217150953_381254N-ACD-BETA',
  '20181206145920_20181218145919_321328N-ACD-BETA',
  '20181206145946_20181218145945_334936N-ACD-BETA',
  '20181206150010_20181218150010_352058N-ACD-BETA',
  '20181206150035_20181218150035_365035N-ACD-BETA',
  '20181206150100_20181218150100_382023N-ACD-BETA',
  '20181207145104_20181219145103_29396N-ACD-BETA',
  '20181207145130_20181219145129_311035N-ACD-BETA',
  '20181207145154_20181219145154_324150N-ACD-BETA',
  '20181207145219_20181219145219_341139N-ACD-BETA',
  '20181207145244_20181219145244_354118N-ACD-BETA',
  '20181207145309_20181219145308_371054N-ACD-BETA',
  '20181209024550_20181221024550_33340N-ACD-BETA',
  '20181209024611_20181221024611_314012N-ACD-BETA',
  '20181209152404_20181221152404_322516N-ACD-BETA',
  '20181209152430_20181221152429_34124N-ACD-BETA',
  '20181210151555_20181222151555_30130N-ACD-BETA',
  '20181210151621_20181222151621_314912N-ACD-BETA',
  '20181210151646_20181222151646_332035N-ACD-BETA',
  '20181210151711_20181222151710_345012N-ACD-BETA',
  '20181210151736_20181222151735_361949N-ACD-BETA',
  '20181211150644_20181223150644_292131N-ACD-BETA',
  '20181211150710_20181223150710_305244N-ACD-BETA',
  '20181211150735_20181223150735_32248N-ACD-BETA',
  '20181211150800_20181223150759_335348N-ACD-BETA',
  '20181211150826_20181223150826_352656N-ACD-BETA',
  '20181211150902_20181223150902_373458N-ACD-BETA',
  '20181212150012_20181224150012_325810N-ACD-BETA',
  '20181212150128_20181224150127_372845N-ACD-BETA',
  '20181213145109_20181225145109_322656N-ACD-BETA',
  '20181213145135_20181225145135_34314N-ACD-BETA',
  '20181213145200_20181225145200_353433N-ACD-BETA',
  '20181213145216_20181225145215_364749N-ACD-BETA',
  '20181217031718_20181229031718_381412N-ACD-BETA',
  '20181217031743_20181229031743_364428N-ACD-BETA',
  '20181217031808_20181229031808_351451N-ACD-BETA',
  '20181217031834_20181229031834_334328N-ACD-BETA',
  '20181217031900_20181229031859_321021N-ACD-BETA',
  '20181218145919_20181230145919_321328N-ACD-BETA',
  '20181218145945_20181230145945_334936N-ACD-BETA',
  '20181218150010_20181230150009_352058N-ACD-BETA',
  '20181218150035_20181230150034_365036N-ACD-BETA',
  '20181218150100_20181230150059_382023N-ACD-BETA',
  '20181219030115_20181231030115_365149N-ACD-BETA',
  '20181219030141_20181231030140_351541N-ACD-BETA',
  '20181219030206_20181231030205_334414N-ACD-BETA',
  '20181219030238_20181231030238_3215N-ACD-BETA',
  '20181220025446_20190101025446_33160N-ACD-BETA',
  '20181221024550_20190102024549_33341N-ACD-BETA',
  '20181221024611_20190102024610_314013N-ACD-BETA',
  '20181221152404_20190102152403_322516N-ACD-BETA',
  '20181221152429_20190102152429_34124N-ACD-BETA',
  '20181222151621_20190103151620_314913N-ACD-BETA',
  '20181222151646_20190103151645_332036N-ACD-BETA',
  '20181222151710_20190103151710_345013N-ACD-BETA',
  '20181223150644_20190104150644_292132N-ACD-BETA',
  '20181223150710_20190104150709_305244N-ACD-BETA',
  '20181223150735_20190104150734_32248N-ACD-BETA',
  '20181223150759_20190104150759_335348N-ACD-BETA',
  '20181223150826_20190104150826_352655N-ACD-BETA',
  '20181223150902_20190104150902_373457N-ACD-BETA',
  '20181224030906_20190105030905_381041N-ACD-BETA',
  '20181224030930_20190105030930_364057N-ACD-BETA',
  '20181224030955_20190105030955_351116N-ACD-BETA',
  '20181224031021_20190105031021_333951N-ACD-BETA',
  '20181225030222_20190106030221_351851N-ACD-BETA',
  '20181225145215_20190106145215_364747N-ACD-BETA',
  '20181226025306_20190107025306_363434N-ACD-BETA',
  '20181226025332_20190107025331_345831N-ACD-BETA',
  '20181226025357_20190107025356_33273N-ACD-BETA',
  '20181226025429_20190107025428_314346N-ACD-BETA',
  '20181227024632_20190108024631_33016N-ACD-BETA',
  '20181227024657_20190108024656_313021N-ACD-BETA',
  '20181228151454_20190109151453_2935N-ACD-BETA',
  '20181228151520_20190109151519_303917N-ACD-BETA',
  '20181228151544_20190109151543_32913N-ACD-BETA',
  '20181228151619_20190109151619_341212N-ACD-BETA',
  '20181228151645_20190109151645_354816N-ACD-BETA',
  '20181229031743_20190110031742_364428N-ACD-BETA',
  '20181229031808_20190110031807_351451N-ACD-BETA',
  '20181229031834_20190110031833_334329N-ACD-BETA',
  '20181229031859_20190110031859_321021N-ACD-BETA',
  '20181230030955_20190111030954_374540N-ACD-BETA',
  '20181230031019_20190111031019_361419N-ACD-BETA',
  '20181230031044_20190111031044_344440N-ACD-BETA',
  '20181230031109_20190111031108_33151N-ACD-BETA',
  '20181230031131_20190111031131_315021N-ACD-BETA',
  '20181230145919_20190111145919_321328N-ACD-BETA',
  '20181230145945_20190111145944_334936N-ACD-BETA',
  '20181230150009_20190111150009_352058N-ACD-BETA',
  '20181230150034_20190111150034_365035N-ACD-BETA',
  '20181230150059_20190111150059_382023N-ACD-BETA',
  '20181231145103_20190112145102_293910N-ACD-BETA',
  '20181231145129_20190112145128_311038N-ACD-BETA',
  '20181231145154_20190112145153_324153N-ACD-BETA',
  '20181231145218_20190112145218_341141N-ACD-BETA',
  '20181231145243_20190112145243_354119N-ACD-BETA',
  '20181231145308_20190112145307_371056N-ACD-BETA']}

In [5]:
os.chdir('C:\\Users\\luker\\Desktop\\d4d_work\\geojsons')
dir_list = os.listdir('.')

target_month_dir = month_file_dict[user_month]

print(user_month + " " + user_day + " is being processed")


apr 21 is being processed


In [6]:
len(target_month_dir)

76

In [7]:
arrival_coords = []
departure_coords = []

for i in range(len(target_month_dir)):
    curr_target_dir = target_month_dir[i]
    print(curr_target_dir)
    curr_target_dir_path = 'C:\\Users\\luker\\Desktop\\d4d_work\\geojsons\\' + curr_target_dir
    os.chdir(curr_target_dir_path)
    file_list = os.listdir('.') 
    
    
    arrivalfname = r""+file_list[0] 
    dfArrival = gpd.read_file(arrivalfname)
    if dfArrival['detect_type'][0] == 'Arrival':
        arrival_date = dfArrival['timestamp'][0]
        arrival_day =  arrival_date[8:10]
        if arrival_day == user_day:
            print("inside arr day")
            point_x = dfArrival['geometry'].x
            point_y = dfArrival['geometry'].y
            for j in range(len(point_x)):
                x = point_x[j]
                y = point_y[j]
                coords = (x,y)
                arrival_coords.append(coords)
            print(arrival_date)
        
        
    departurefname = r""+file_list[1]      
    dfDeparture = gpd.read_file(departurefname)
    if dfDeparture['detect_type'][0] == 'Departure':
        departure_date = dfDeparture['timestamp'][0]
        departure_day =  departure_date[8:10]
        if departure_day == user_day:
            print("inside dep day")
            point_x = dfDeparture['geometry'].x
            point_y = dfDeparture['geometry'].y
            for j in range(len(point_x)):
                x = point_x[j]
                y = point_y[j]
                coords = (x,y)
                departure_coords.append(coords)
            print(departure_date)


20190320144244_20190401144245_292628N-ACD-BETA
20190320144310_20190401144310_305757N-ACD-BETA
20190321152406_20190402152406_30129N-ACD-BETA
inside dep day
2019-03-21T00:00:00
20190321152431_20190402152431_314335N-ACD-BETA
inside dep day
2019-03-21T00:00:00
20190321152456_20190402152456_331316N-ACD-BETA
inside dep day
2019-03-21T00:00:00
20190321152520_20190402152521_344254N-ACD-BETA
inside dep day
2019-03-21T00:00:00
20190322151452_20190403151453_2933N-ACD-BETA
20190322151518_20190403151518_303915N-ACD-BETA
20190322151542_20190403151542_32910N-ACD-BETA
20190322151618_20190403151618_341210N-ACD-BETA
20190323150811_20190404150811_32611N-ACD-BETA
20190323150837_20190404150837_334220N-ACD-BETA
20190323150902_20190404150902_351340N-ACD-BETA
20190323150926_20190404150927_364317N-ACD-BETA
20190323150951_20190404150952_381253N-ACD-BETA
20190324145918_20190405145918_321325N-ACD-BETA
20190324145943_20190405145944_334933N-ACD-BETA
20190324150033_20190405150033_365032N-ACD-BETA
20190324150058_2019

In [9]:
max_x_arr = 0
min_x_arr = 50
max_y_arr = 0
min_y_arr = 50
for i in range(len(arrival_coords)):
    x = arrival_coords[i][0]
    if x > max_x_arr:
        max_x_arr = x
    if x < min_x_arr:
        min_x_arr = x
    y = arrival_coords[i][1]
    if y > max_y_arr:
        max_y_arr = y
    if y < min_y_arr:
        min_y_arr = y
print(max_x_arr) 
print(min_x_arr) 
print(max_y_arr) 
print(min_y_arr) 


'''for i in range(len(arrival_coords)):
    j=0
    while True:
    '''

51.5002139
36.76191
35.922763
29.296726


'for i in range(len(arrival_coords)):\n    j=0\n    while True:\n    '

In [ ]:
arrival_coords
#Length Of A Degree Of Longitude In Meters

In [10]:
max_x_dep = 0
min_x_dep = 50
max_y_dep = 0
min_y_dep = 50
arrival_coords_x = []
for i in range(len(departure_coords)):
    x = departure_coords[i][0]
    arrival_coords_x.append(x)
    if x > max_x_dep:
        max_x_dep = x
    if x < min_x_dep:
        min_x_dep = x
    y = departure_coords[i][1]
    if y > max_y_dep:
        max_y_dep = y
    if y < min_y_dep:
        min_y_dep = y
print(max_x_dep) 
print(min_x_dep) 
print(max_y_dep) 
print(min_y_dep) 

51.4997002
37.4377106
37.225876
29.1642754


In [11]:
print(len(arrival_coords_x))

6540


In [12]:
#X = np.array([arrival_coords_x])

#kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

#kmeans.labels_

#kmeans.predict([[43]])

#kmeans.cluster_centers_


In [13]:
len(departure_coords)

6540

In [14]:
os.chdir('C:\\Users\\luker\\Desktop\\d4d_work')

#create map object
m = folium.Map(location=[33.3152, 44.3661], zoom_start=12)

#global tooltip
#tooltip = 'Click for more Info'

#create custom marker icon
#logoIcon = folium.features.CustomIcon('logo.png', icon_size=(50,50))


for i in range(len(arrival_coords)):
    x = arrival_coords[i][0]
    y = arrival_coords[i][1]
    folium.Marker([y, x], 
              popup='<strong>arrival</strong>',
              #tooltip=tooltip,
              icon=folium.Icon(color='green',icon='leaf',icon_size=(15,15))).add_to(m),
    
for i in range(len(departure_coords)):
    x = departure_coords[i][0]
    y = departure_coords[i][1]
    folium.Marker([y, x], 
              popup='<strong>Departure</strong>',
              #tooltip=tooltip,
              icon=folium.Icon(color='red',icon='transit_enterexit',icon_size=(15,15))).add_to(m),    
'''    
# Circle Marker
folium.CircleMarker(
    location=[33.280888, 44.232883],
    radius=50,
    popup='hot zone',
    color='#428bca',
    fill=True,
    fill_color='#428bca'
)add_to(m)

'''                                     
#generate map
m.save('map.html')

#todo auto map.html